In [44]:
import json

PATH = json.load(open('../path_config.json'))

In [45]:
test_source_path = PATH['xsum_fariseq'] + '/test.source'
test_target_path = PATH['xsum_fariseq'] + '/test.target'
xsum_test_prediction = '../preds/xsum_official.hypo'

In [46]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [47]:
xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)
xsum_preds = read_lines(xsum_test_prediction)
assert len(xsum_source) == len(xsum_target) == len(xsum_preds)
print(len(xsum_source))

11301


#### Annotation

In [48]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [49]:
# import nltk

# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag

In [50]:
ex = 'Middlesbrough midfielder Federico Fazio has left the Championship club after his contract was terminated by mutual consent.'

In [51]:
# ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(xsum_preds[ID])))
# # print(ne_tree)

#### Build Dataset

In [52]:
import json
import copy

from random import randint

In [53]:
# format:
# {
#     'id': 0,
#     'pred': 'This is a test.',
#     'ents': [{'start': 62,official_predsnd': 75, 'type': 'CARDINAL', 'ent': 'European', 'label': 0},
#              {'start': 91, 'end': 93, 'type': 'CARDINAL', 'ent': 'European', 'label': 0},
#              {'start': 109, 'end': 127, 'type': 'CARDINAL', 'ent': 'European', 'label': 0}
#             ],
#     'hallucinations': ['more than 100 jobs']
# }
# LABEL: 
# 0 non-hallucination
# 1 true hallucination
# 2 false hallucination

annotated = [
{'id': 8805,
 'pred': "A Wigan-based biscuit maker has gone into administration with the loss of more than 100 jobs following the UK's vote to leave the EU.",
 'ents': [{'start': 2, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Wigan'},
  {'start': 74,
   'end': 87,
   'label': 2,
   'type': 'CARDINAL',
   'ent': 'more than 100'},
  {'start': 107, 'end': 109, 'label': 1, 'type': 'GPE', 'ent': 'UK'},
  {'start': 130, 'end': 132, 'label': 0, 'type': 'ORG', 'ent': 'EU'}],
 'hallucinations': []
},
{'id': 9444,
 'pred': 'Twin-to-twin transfusion syndrome (TTTS) is being tracked by a hospital in Cardiff in a bid to save the lives of babies born with the condition.',
 'ents': [{'start': 35, 'end': 39, 'label': 0, 'type': 'ORG', 'ent': 'TTTS'},
          {'start': 75, 'end': 82, 'label': 2, 'type': 'LOC', 'ent': 'Cardiff'}],
 'hallucinations': ['in Cardiff']
},
{'id': 7079,
 'pred': "New Celtic manager Brendan Rodgers has met the club's captain for the first time as he prepares for his first game in charge of the club.",
 'ents': [{'start': 0, 'end': 10, 'label': 1, 'type': 'ORG', 'ent': 'New Celtic'},
          {'start': 19, 'end': 26, 'label': 1, 'type': 'PERSON', 'ent': 'Brendan'},
          {'start': 27, 'end': 34, 'label': 0, 'type': 'PERSON', 'ent': 'Rodgers'},
          {'start': 70, 'end': 75, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 104, 'end': 109, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'}],
 'hallucinations': ['New Celtic', 'Brendan']
},
{'id': 2427,
 'pred': 'The Catholic Church should apologise to the families of unwed mothers who died at mother-and-baby homes, the Archbishop of Dublin, Rowan Martin, has said.',
 'ents': [{'start': 0, 'end': 19, 'label': 0, 'type': 'ORG', 'ent': 'The Catholic Church'},
          {'start': 123, 'end': 129, 'label': 2, 'type': 'GPE', 'ent': 'Dublin'},
          {'start': 131, 'end': 136, 'label': 2, 'type': 'PERSON', 'ent': 'Rowan'},
          {'start': 137, 'end': 143, 'label': 0, 'type': 'PERSON', 'ent': 'Martin'}],
 'hallucinations': ['Dublin', 'Rowan']
},
{'id': 6220,
 'pred': 'The body of a man whose body was found at the site of the Swansea Bay Power Station collapse has been removed from the site.',
 'ents': [{'start': 58, 'end': 69, 'label': 3, 'type': 'GPE', 'ent': 'Swansea Bay'}],
 'hallucinations': ['Swansea Bay Power Station']
},
{'id': 5860,
 'pred': 'The government has announced a voluntary higher rate of the London living wage, which will rise to £9.40 an hour from 1 July.',
 'ents': [{'start': 60, 'end': 66, 'label': 0, 'type': 'GPE', 'ent': 'London'},
          {'start': 100, 'end': 104, 'label': 0, 'type': 'MONEY', 'ent': '9.40'},
          {'start': 118, 'end': 124, 'label': 2, 'type': 'DATE', 'ent': '1 July'}],
 'hallucinations': ['from 1 July']
},
{'id': 10579,
 'pred': 'Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed by a gunman in the Australian city.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'ORG',
   'ent': 'Sydney'},
  {'start': 22, 'end': 27, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
  {'start': 60, 'end': 68, 'label': 2, 'type': 'ORG', 'ent': 'Waverley'},
  {'start': 83, 'end': 86, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
  {'start': 124,
   'end': 134,
   'label': 0,
   'type': 'NORP',
   'ent': 'Australian'}],
 'hallucinations': []},
{'id': 5455,
 'pred': 'As Abid Naseer went on trial in the United States for his role in an al-Qaeda terror plot to carry out a "mass-casualty" bomb attack in Manchester, BBC News looks at what police believe would have happened if he had not been caught.',
 'ents': [{'start': 3, 'end': 7, 'label': 0, 'type': 'PERSON', 'ent': 'Abid'},
          {'start': 8, 'end': 14, 'label': 0, 'type': 'PERSON', 'ent': 'Naseer'},
          {'start': 32, 'end': 49, 'label': 0, 'type': 'GPE', 'ent': 'the United States'},
          {'start': 69, 'end': 77, 'label': 0, 'type': 'ORG', 'ent': 'al-Qaeda'},
          {'start': 136, 'end': 146, 'label': 0, 'type': 'GPE', 'ent': 'Manchester'},
          {'start': 148, 'end': 156, 'label': 1, 'type': 'ORG', 'ent': 'BBC News'}],
 'hallucinations': []
},
{'id': 2407,
 'pred': "The world's biggest asset manager, BlackRock, is planning to cut up to 1,000 jobs as part of a cost-cutting drive, according to reports.",
 'ents': [{'start': 35, 'end': 44, 'label': 0, 'type': 'ORG', 'ent': 'BlackRock'},
          {'start': 65, 'end': 76, 'label': 2, 'type': 'CARDINAL', 'ent': 'up to 1,000'}],
 'hallucinations': ['up to 1,000 jobs']
},
{'id': 416,
 'pred': 'Crawley Town have signed Dutch striker Marvin Verheydt from Dutch club Maastricht for an undisclosed fee, subject to international clearance.',
 'ents': [{'start': 0, 'end': 12, 'label': 1, 'type': 'ORG', 'ent': 'Crawley Town'},
          {'start': 25, 'end': 30, 'label': 1, 'type': 'GPE', 'ent': 'Dutch'},
          {'start': 39, 'end': 45, 'label': 2, 'type': 'PERSON', 'ent': 'Marvin'},
          {'start': 46, 'end': 54, 'label': 0, 'type': 'PERSON', 'ent': 'Verheydt'},
          {'start': 60, 'end': 65, 'label': 1, 'type': 'GPE', 'ent': 'Dutch'},
          {'start': 71, 'end': 81, 'label': 0, 'type': 'ORG', 'ent': 'Maastricht'}],
 'hallucinations': []
},
{'id': 6824,
 'pred': 'Leigh Centurions have agreed a deal to sign Salford Red Devils half-back Ryan Chase on a free transfer after he was cleared to leave the club following a disciplinary hearing.',
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'ORG', 'ent': 'Leigh'},
          {'start': 6, 'end': 16, 'label': 1, 'type': 'ORG', 'ent': 'Centurions'},
          {'start': 44, 'end': 51, 'label': 0, 'type': 'GPE', 'ent': 'Salford'},
          {'start': 52, 'end': 62, 'label': 0, 'type': 'ORG', 'ent': 'Red Devils'},
          {'start': 73, 'end': 77, 'label': 2, 'type': 'PERSON', 'ent': 'Ryan'},
          {'start': 78, 'end': 83, 'label': 0, 'type': 'PERSON', 'ent': 'Chase'}],
 'hallucinations': ['Centurions', 'Ryan']
},
{'id': 1513,
 'pred': 'Manchester United have failed to complete the signing of Spain goalkeeper David de Gea from Manchester United to Real Madrid in a deal that was due to be completed on Monday night, but the transfer window is still open in Spain.',
 'ents': [{'start': 0, 'end': 17, 'label': 1, 'type': 'ORG', 'ent': 'Manchester United'},
          {'start': 57, 'end': 62, 'label': 0, 'type': 'GPE', 'ent': 'Spain'},
          {'start': 74, 'end': 79, 'label': 1, 'type': 'PERSON', 'ent': 'David'},
          {'start': 80, 'end': 82, 'label': 0, 'type': 'PERSON', 'ent': 'de'},
          {'start': 83, 'end': 86, 'label': 0, 'type': 'PERSON', 'ent': 'Gea'},
          {'start': 92, 'end': 109, 'label': 1, 'type': 'ORG', 'ent': 'Manchester United'},
          {'start': 113, 'end': 124, 'label': 0, 'type': 'ORG', 'ent': 'Real Madrid'},
          {'start': 167, 'end': 179, 'label': 2, 'type': 'TIME', 'ent': 'Monday night'},
          {'start': 222, 'end': 227, 'label': 0, 'type': 'GPE', 'ent': 'Spain'}],
 'hallucinations': []
},
{'id': 1819,
 'pred': 'The brother of one of the victims of Bloody Sunday has said plans to hold a march by former soldiers in Londonderry are "totally insensitive" to the people of the city.',
 'ents': [{'start': 15, 'end': 18, 'label': 0, 'type': 'CARDINAL', 'ent': 'one'},
          {'start': 37, 'end': 50, 'label': 0, 'type': 'EVENT', 'ent': 'Bloody Sunday'},
          {'start': 104, 'end': 115, 'label': 1, 'type': 'GPE', 'ent': 'Londonderry'}],
 'hallucinations': []
},
{'id': 560,
 'pred': 'South Africa will appoint a new head coach ahead of their tour of England next month after the end of the current one-day international and Test series against Sri Lanka, reports BBC Sport.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'GPE', 'ent': 'South Africa'},
          {'start': 66, 'end': 73, 'label': 0, 'type': 'GPE', 'ent': 'England'},
          {'start': 74, 'end': 84, 'label': 1, 'type': 'DATE', 'ent': 'next month'},
          {'start': 114, 'end': 121, 'label': 0, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 160, 'end': 169, 'label': 0, 'type': 'GPE', 'ent': 'Sri Lanka'},
          {'start': 179, 'end': 188, 'label': 1, 'type': 'ORG', 'ent': 'BBC Sport'}],
 'hallucinations': ['next month', 'reports BBC Sport']
},
{'id': 4174,
 'pred': 'A British woman who was jailed in Malaysia for indecency after stripping off in public at a popular tourist destination has left the country.',
 'ents': [{'start': 2,
   'end': 9,
   'label': 1,
   'type': 'NORP',
   'ent': 'British'},
  {'start': 34, 'end': 42, 'label': 0, 'type': 'GPE', 'ent': 'Malaysia'}],
 'hallucinations': []},
{'id': 2194,
 'pred': '"I\'m a very good storyteller," says David Chase, as he tells the story of his life.',
 'ents': [{'start': 36, 'end': 41, 'label': 2, 'type': 'PERSON', 'ent': 'David'},
          {'start': 42, 'end': 47, 'label': 0, 'type': 'PERSON', 'ent': 'Chase'}],
 'hallucinations': []
},
{'id': 2517,
 'pred': "Saturday's League One game between Oldham Athletic and Bolton Wanderers has been postponed because of a waterlogged pitch.",
 'ents': [{'start': 0, 'end': 8, 'label': 1, 'type': 'DATE', 'ent': 'Saturday'},
          {'start': 18, 'end': 21, 'label': 1, 'type': 'CARDINAL', 'ent': 'One'},
          {'start': 35, 'end': 50, 'label': 1, 'type': 'ORG', 'ent': 'Oldham Athletic'},
          {'start': 55, 'end': 71, 'label': 2, 'type': 'ORG', 'ent': 'Bolton Wanderers'}],
 'hallucinations': ["Saturday's League One", 'Oldham Athletic', 'Bolton Wanderers']
},
{'id': 922,
 'pred': 'The way GCSEs are graded in England has failed to meet five conditions set out by the exams watchdog Ofqual, according to a new report.',
 'ents': [{'start': 28, 'end': 35, 'label': 1, 'type': 'GPE', 'ent': 'England'},
          {'start': 55, 'end': 59, 'label': 0, 'type': 'CARDINAL', 'ent': 'five'},
          {'start': 101, 'end': 107, 'label': 0, 'type': 'ORG', 'ent': 'Ofqual'}],
 'hallucinations': []
},
{'id': 10066,
 'pred': 'Police investigating the disappearance of a man who went missing in May have launched a national appeal for information about his disappearance.',
 'ents': [{'start': 68, 'end': 71, 'label': 0, 'type': 'DATE', 'ent': 'May'}],
 'hallucinations': []
},
{'id': 1367,
 'pred': 'Jockey David Mullins rode Rule The World to victory in the Grand National at Aintree for the first time on Saturday.',
 'ents': [{'start': 7, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': 'David'},
  {'start': 13, 'end': 20, 'label': 0, 'type': 'PERSON', 'ent': 'Mullins'},
  {'start': 55,
   'end': 73,
   'label': 0,
   'type': 'FAC',
   'ent': 'the Grand National'},
  {'start': 77, 'end': 84, 'label': 0, 'type': 'FAC', 'ent': 'Aintree'},
  {'start': 93, 'end': 98, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
  {'start': 107,
   'end': 115,
   'label': 0,
   'type': 'DATE',
   'ent': 'Saturday'}],
 'hallucinations': []},
{'id': 11207,
 'pred': 'The jury in the trial of the man accused of carrying out the Boston Marathon bombings has heard graphic testimony about the injuries suffered by the victims of the attack.',
 'ents': [{'start': 61, 'end': 76, 'label': 1, 'type': 'ORG', 'ent': 'Boston Marathon'}],
 'hallucinations': []
},
{'id': 3211,
 'pred': 'A man has been charged in the US state of Michigan after a student with a severe peanut allergy was allegedly forced to eat peanut butter as part of a fraternity initiation ritual.',
 'ents': [{'start': 30, 'end': 32, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 42, 'end': 50, 'label': 0, 'type': 'GPE', 'ent': 'Michigan'}],
 'hallucinations': []
},
{'id': 10524,
 'pred': "Edinburgh Zoo's giant panda Tian Tian is expected to give birth at the end of the month, the zoo has said.",
 'ents': [{'start': 0, 'end': 13, 'label': 0, 'type': 'ORG', 'ent': "Edinburgh Zoo"},
          {'start': 28, 'end': 32, 'label': 0, 'type': 'PERSON', 'ent': 'Tian'},
          {'start': 33, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'Tian'},
          {'start': 67, 'end': 87, 'label': 0, 'type': 'DATE', 'ent': 'the end of the month'}],
 'hallucinations': []
},
{'id': 10488,
 'pred': 'Pupils who are able and talented are not being given enough support in the education system, according to the chief inspector of schools at Estyn.',
 'ents': [{'start': 140, 'end': 145, 'label': 0, 'type': 'LOC', 'ent': 'Estyn'}],
 'hallucinations': []
},
{'id': 4809,
 'pred': 'US Secretary of State Rex Tillerson has said he was "convinced" by his wife to take up the job by President Donald Trump, who asked him to be his secretary of state.',
 'ents': [{'start': 0, 'end': 2, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 22, 'end': 25, 'label': 1, 'type': 'PERSON', 'ent': 'Rex'},
          {'start': 26, 'end': 35, 'label': 0, 'type': 'PERSON', 'ent': 'Tillerson'},
          {'start': 108, 'end': 114, 'label': 0, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 115, 'end': 120, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'}],
 'hallucinations': []
},
{'id': 6228,
 'pred': 'A group of MPs has called on the government to legalise medical cannabis after a study found that one million people across the UK rely on the drug for medical reasons, but there is limited or no convincing evidence that it works.',
 'ents': [{'start': 98, 'end': 109, 'label': 0, 'type': 'CARDINAL', 'ent': 'one million'},
          {'start': 128, 'end': 130, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []
},
{'id': 10517,
 'pred': "The police ombudsman's office is to resume investigations into more than 100 historical complaints against the PSNI, it has been announced.",
 'ents': [{'start': 63,
   'end': 76,
   'label': 2,
   'type': 'CARDINAL',
   'ent': 'more than 100'},
  {'start': 111, 'end': 115, 'label': 0, 'type': 'ORG', 'ent': 'PSNI'}],
 'hallucinations': []},
{'id': 7247,
 'pred': 'Dundee Stars head coach Marc LeFebvre hopes his side can learn from their experience at the Elite League play-off finals and return to the finals next season.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'ORG', 'ent': 'Dundee Stars'},
          {'start': 24, 'end': 28, 'label': 1, 'type': 'PERSON', 'ent': 'Marc'},
          {'start': 29, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'LeFebvre'},
          {'start': 146, 'end': 157, 'label': 0, 'type': 'DATE', 'ent': 'next season'}],
 'hallucinations': []
},
{'id': 2724,
 'pred': 'Cardiff and its city region should be given more power by the Welsh assembly to work better with the "Northern Powerhouse" region of England, the chairman of the Northern Powerhouse group has said.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Cardiff'},
          {'start': 62, 'end': 67, 'label': 0, 'type': 'GPE', 'ent': 'Welsh'},
          {'start': 133, 'end': 140, 'label': 0, 'type': 'GPE', 'ent': 'England'},
          {'start': 162, 'end': 181, 'label': 0, 'type': 'ORG', 'ent': 'Northern Powerhouse'}],
 'hallucinations': []
},
{'id': 4630,
 'pred': 'Paralympic gold medallist Bethany Brown says she will not change her bow to recurve in order to qualify for the 2016 Rio Games.',
 'ents': [{'start': 26, 'end': 33, 'label': 2, 'type': 'PERSON', 'ent': 'Bethany'},
          {'start': 34, 'end': 39, 'label': 0, 'type': 'PERSON', 'ent': 'Brown'},
          {'start': 112, 'end': 116, 'label': 0, 'type': 'DATE', 'ent': '2016'}],
 'hallucinations': ['Bethany']
},
{'id': 6126,
 'pred': 'The Supreme Court has ruled that the assembly has the power to set minimum wages for agricultural workers in Wales.',
 'ents': [{'start': 0, 'end': 17, 'label': 0, 'type': 'ORG', 'ent': 'The Supreme Court'},
          {'start': 109, 'end': 114, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': []
},
{'id': 3529,
 'pred': 'The daughter of the first baby to be born on an air ambulance has joined Loganair as a cabin crew attendant, 40 years after her birth.',
 'ents': [{'start': 20, 'end': 25, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 73, 'end': 81, 'label': 0, 'type': 'PERSON', 'ent': 'Loganair'},
          {'start': 109, 'end': 117, 'label': 0, 'type': 'DATE', 'ent': '40 years'}],
 'hallucinations': []
},
{'id': 11006,
 'pred': 'Bristol City moved three points off the Championship play-off places with a hard-fought win over Ipswich Town at Ashton Gate.',
 'ents': [{'start': 0,
   'end': 12,
   'label': 0,
   'type': 'GPE',
   'ent': 'Bristol City'},
  {'start': 19, 'end': 24, 'label': 3, 'type': 'CARDINAL', 'ent': 'three'},
  {'start': 97,
   'end': 109,
   'label': 0,
   'type': 'FAC',
   'ent': 'Ipswich Town'},
  {'start': 113,
   'end': 124,
   'label': 0,
   'type': 'FAC',
   'ent': 'Ashton Gate'}],
 'hallucinations': []},
{'id': 8447,
 'pred': 'The White House has confirmed that President Barack Obama and First Lady Michelle Obama were among the celebrities who attended a private party in New York to celebrate their wedding anniversary.',
 'ents': [{'start': 0, 'end': 15, 'label': 1, 'type': 'ORG', 'ent': 'The White House'},
          {'start': 45, 'end': 51, 'label': 1, 'type': 'PERSON', 'ent': 'Barack'},
          {'start': 52, 'end': 57, 'label': 0, 'type': 'PERSON', 'ent': 'Obama'},
          {'start': 73, 'end': 81, 'label': 1, 'type': 'PERSON', 'ent': 'Michelle'},
          {'start': 82, 'end': 87, 'label': 0, 'type': 'PERSON', 'ent': 'Obama'},
          {'start': 147, 'end': 155, 'label': 3, 'type': 'GPE', 'ent': 'New York'}],
 'hallucinations': ['The White House', 'Barack', 'First Lady Michelle', 'to celebrate their wedding anniversary']
},
{'id': 9980,
 'pred': "Tottenham Hotspur have agreed to pay £1.5m to the family of a 17-year-old boy who suffered brain injuries after a heart attack while playing for the club's youth academy.",
 'ents': [{'start': 0,
   'end': 9,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Tottenham'},
  {'start': 10, 'end': 17, 'label': 1, 'type': 'PERSON', 'ent': 'Hotspur'},
  {'start': 38, 'end': 41, 'label': 2, 'type': 'MONEY', 'ent': '1.5'}],
 'hallucinations': []},
{'id': 8685,
 'pred': 'League One side Wigan Athletic have signed Bournemouth midfielder Chris MacDonald on loan until the end of the season.',
 'ents': [{'start': 7, 'end': 10, 'label': 0, 'type': 'CARDINAL', 'ent': 'One'},
          {'start': 16, 'end': 30, 'label': 1, 'type': 'ORG', 'ent': 'Wigan Athletic'},
          {'start': 43, 'end': 54, 'label': 1, 'type': 'LOC', 'ent': 'Bournemouth'},
          {'start': 66, 'end': 71, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
          {'start': 72, 'end': 81, 'label': 0, 'type': 'PERSON', 'ent': 'MacDonald'},
          {'start': 96, 'end': 117, 'label': 2, 'type': 'DATE', 'ent': 'the end of the season'}],
 'hallucinations': ['Wigan Athletic', 'Bournemouth midfielder', 'Chris', 'on loan until the end of the season']
},
{'id': 11080,
 'pred': 'The new head of the Public Services Complaints Commission for Wales has been appointed.',
 'ents': [{'start': 16, 'end': 57, 'label': 2, 'type': 'ORG', 'ent': 'the Public Services Complaints Commission'},
          {'start': 62, 'end': 67, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': ['the Public Services Complaints Commission']
},
{'id': 8558,
 'pred': 'A former Welsh Secretary has accused the UK government of "breaking a manifesto commitment" by removing the requirement for a referendum on income tax powers being devolved to the assembly.',
 'ents': [{'start': 9,
   'end': 14,
   'label': 0,
   'type': 'ORG',
   'ent': 'Welsh'},
  {'start': 41, 'end': 43, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []},
{'id': 6931,
 'pred': "Indian Prime Minister Narendra Modi's visit to Silicon Valley is the latest in a series of high-profile visits to the US, where he's met some of the country's best-known and most successful technology leaders, many of whom have lived and worked in the region for decades.",
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'NORP',
   'ent': 'Indian'},
  {'start': 22, 'end': 30, 'label': 1, 'type': 'PERSON', 'ent': 'Narendra'},
  {'start': 31, 'end': 35, 'label': 0, 'type': 'PERSON', 'ent': 'Modi'},
  {'start': 47,
   'end': 61,
   'label': 0,
   'type': 'LOC',
   'ent': 'Silicon Valley'},
  {'start': 118, 'end': 120, 'label': 0, 'type': 'GPE', 'ent': 'US'}],
 'hallucinations': []},
{'id': 6373,
 'pred': 'Police investigating the disappearance of an RAF serviceman who went missing after a night out in Bury St Edmunds have released CCTV images of three teenagers they want to trace.',
 'ents': [{'start': 45, 'end': 48, 'label': 0, 'type': 'ORG', 'ent': 'RAF'},
          {'start': 83, 'end': 90, 'label': 0, 'type': 'TIME', 'ent': 'a night'},
          {'start': 98, 'end': 113, 'label': 0, 'type': 'GPE', 'ent': 'Bury St Edmunds'},
          {'start': 128, 'end': 132, 'label': 0, 'type': 'ORG', 'ent': 'CCTV'},
          {'start': 143, 'end': 148, 'label': 0, 'type': 'CARDINAL', 'ent': 'three'}],
 'hallucinations': []
},
{'id': 9961,
 'pred': 'Three Syrian refugees who overpowered a suspected terror suspect in the German city of Leipzig and tied him up have been hailed as "heroes" by the country\'s media and politicians.',
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'CARDINAL', 'ent': 'Three'},
          {'start': 6, 'end': 12, 'label': 0, 'type': 'NORP', 'ent': 'Syrian'},
          {'start': 72, 'end': 78, 'label': 0, 'type': 'NORP', 'ent': 'German'},
          {'start': 87, 'end': 94, 'label': 0, 'type': 'GPE', 'ent': 'Leipzig'}],
 'hallucinations': []
},
{'id': 6235,
 'pred': "China's Tiangong-1 space station will burn up when it re-enters the Earth's atmosphere later this week, according to the country's space agency.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'GPE', 'ent': 'China'},
          {'start': 8, 'end': 18, 'label': 0, 'type': 'FAC', 'ent': 'Tiangong-1'},
          {'start': 87, 'end': 102, 'label': 2, 'type': 'DATE', 'ent': 'later this week'}],
 'hallucinations': []
},
{'id': 5414,
 'pred': 'Chinese search giant Baidu has unveiled a voice-activated artificial intelligence (AI) that can be used on a smart speaker.',
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'NORP',
   'ent': 'Chinese'},
  {'start': 21, 'end': 26, 'label': 0, 'type': 'ORG', 'ent': 'Baidu'}],
 'hallucinations': []
},
{'id': 1168,
 'pred': 'Belfast-based Mexican restaurant chain Boojum has been sold for an undisclosed sum to a group of investors in the Republic of Ireland.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'GPE', 'ent': 'Belfast'},
          {'start': 14, 'end': 21, 'label': 0, 'type': 'NORP', 'ent': 'Mexican'},
          {'start': 39, 'end': 45, 'label': 0, 'type': 'PERSON', 'ent': 'Boojum'},
          {'start': 110, 'end': 133, 'label': 1, 'type': 'GPE', 'ent': 'the Republic of Ireland'}],
 'hallucinations': []
},
{'id': 7987,
 'pred': "The Northern Ireland Chamber of Commerce has said that the renegotiation of the UK's relationship with the European Union (EU) is having little impact on day-to-day business in Northern Ireland.",
 'ents': [{'start': 4, 'end': 20, 'label': 0, 'type': 'NORP', 'ent': 'Northern Ireland'},
          {'start': 21, 'end': 40, 'label': 0, 'type': 'ORG', 'ent': 'Chamber of Commerce'},
          {'start': 80, 'end': 82, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 103, 'end': 121, 'label': 0, 'type': 'ORG', 'ent': 'the European Union'},
          {'start': 123, 'end': 125, 'label': 0, 'type': 'ORG', 'ent': 'EU'},
          {'start': 177, 'end': 193, 'label': 0, 'type': 'GPE', 'ent': 'Northern Ireland'}],
 'hallucinations': []
},
{'id': 9128,
 'pred': 'More than a dozen teachers in Wales have been banned from the profession after being found guilty of posting sexual content on social media, figures have shown.',
 'ents': [{'start': 0,
   'end': 17,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'More than a dozen'},
  {'start': 30, 'end': 35, 'label': 1, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': []},
{'id': 8495,
 'pred': 'British pair Naomi Broady and Heather Watson were beaten 6-3 6-2 by Chan Yung-jan and Chan Shuai-Wei in the doubles final of the Hong Kong Open.',
 'ents': [{'start': 0, 'end': 7, 'label': 1, 'type': 'NORP', 'ent': 'British'},
          {'start': 13, 'end': 18, 'label': 1, 'type': 'PERSON', 'ent': 'Naomi'},
          {'start': 19, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Broady'},
          {'start': 30, 'end': 37, 'label': 1, 'type': 'PERSON', 'ent': 'Heather'},
          {'start': 38, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'Watson'},
          {'start': 57, 'end': 60, 'label': 1, 'type': 'CARDINAL', 'ent': '6-3'},
          {'start': 61, 'end': 64, 'label': 3, 'type': 'CARDINAL', 'ent': '6-2'},
          {'start': 68, 'end': 72, 'label': 0, 'type': 'PERSON', 'ent': 'Chan'},
          {'start': 73, 'end': 81, 'label': 1, 'type': 'PERSON', 'ent': 'Yung-jan'},
          {'start': 86, 'end': 90, 'label': 0, 'type': 'PERSON', 'ent': 'Chan'},
          {'start': 91, 'end': 100, 'label': 2, 'type': 'PERSON', 'ent': 'Shuai-Wei'},
          {'start': 125, 'end': 143, 'label': 1, 'type': 'ORG', 'ent': 'the Hong Kong Open'}],
 'hallucinations': ['British', 'Naomi', 'Heather', '6-3 6-2', 'Yung-jan', 'Shuai-Wei', 'the Hong Kong Open']
},
{'id': 6486,
 'pred': 'North Korean leader Kim Jong-un has threatened to launch a "fire and fury" missile attack on the US island of Guam, in what has been described as a warning to President Donald Trump.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'NORP', 'ent': 'North Korean'},
          {'start': 20, 'end': 23, 'label': 0, 'type': 'PERSON', 'ent': 'Kim'},
          {'start': 24, 'end': 31, 'label': 0, 'type': 'PERSON', 'ent': 'Jong-un'},
          {'start': 97, 'end': 99, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 110, 'end': 114, 'label': 0, 'type': 'GPE', 'ent': 'Guam'},
          {'start': 169, 'end': 175, 'label': 1, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 176, 'end': 181, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'}],
 'hallucinations': []
},
{'id': 1584,
 'pred': 'Northampton Town have been placed into administration by Northampton Borough Council.',
 'ents': [{'start': 0, 'end': 11, 'label': 0, 'type': 'GPE', 'ent': 'Northampton'},
          {'start': 57, 'end': 84, 'label': 0, 'type': 'ORG', 'ent': 'Northampton Borough Council'}],
 'hallucinations': []
},
{'id': 10958,
 'pred': 'Police have said the death of a man at a hotel in Edinburgh is not being treated as suspicious after his body was found.',
 'ents': [{'start': 50,
   'end': 59,
   'label': 1,
   'type': 'GPE',
   'ent': 'Edinburgh'}],
 'hallucinations': ['in Edinburgh']
},
{'id': 11017,
 'pred': 'The RSPCA has called on Wrexham council to drop plans to ban dogs from play areas and sports pitches, saying it could punish responsible dog owners.',
 'ents': [{'start': 4, 'end': 9, 'label': 0, 'type': 'ORG', 'ent': 'RSPCA'},
          {'start': 24, 'end': 31, 'label': 0, 'type': 'GPE', 'ent': 'Wrexham'}],
 'hallucinations': []
},
{'id': 11037,
 'pred': 'A buzzard has been released back into the wild after becoming stuck in a fence during a chase in Cheshire.',
 'ents': [{'start': 97, 'end': 105, 'label': 2, 'type': 'GPE', 'ent': 'Cheshire'}],
 'hallucinations': ['in Cheshire']
},
{'id': 5148,
 'pred': 'Five children at a Neath Port Talbot nursery have been diagnosed with measles, health officials have said.',
 'ents': [{'start': 0, 'end': 4, 'label': 3, 'type': 'CARDINAL', 'ent': 'Five'},
          {'start': 19, 'end': 24, 'label': 0, 'type': 'LOC', 'ent': 'Neath'},
          {'start': 25, 'end': 36, 'label': 1, 'type': 'GPE', 'ent': 'Port Talbot'}],
 'hallucinations': ['Port Talbot']
},

{'id': 10380,
 'pred': 'A Palestinian woman who was seriously injured in a firebombing in the occupied West Bank that killed her toddler son and her husband has died in hospital, her family says.',
 'ents': [{'start': 2, 'end': 13, 'label': 0, 'type': 'NORP', 'ent': 'Palestinian'},
          {'start': 79, 'end': 88, 'label': 0, 'type': 'GPE', 'ent': 'West Bank'}],
 'hallucinations': ['in a firebombing', 'her family says']
},
{'id': 5747,
 'pred': 'Six teenagers have died from carbon monoxide poisoning after a party at a remote hut in northern Germany, police say.',
 'ents': [{'start': 0, 'end': 3, 'label': 0, 'type': 'CARDINAL', 'ent': 'Six'},
          {'start': 97, 'end': 104, 'label': 0, 'type': 'GPE', 'ent': 'Germany'}],
 'hallucinations': []
},
{'id': 10943,
 'pred': "A powerful cyclone has killed at least 11 people and injured more than 100 in Vanuatu, the Pacific nation's president has said.",
 'ents': [{'start': 30, 'end': 41, 'label': 2, 'type': 'CARDINAL', 'ent': 'at least 11'},
          {'start': 61, 'end': 74, 'label': 2, 'type': 'CARDINAL', 'ent': 'more than 100'},
          {'start': 78, 'end': 85, 'label': 0, 'type': 'GPE', 'ent': 'Vanuatu'},
          {'start': 91, 'end': 98, 'label': 1, 'type': 'LOC', 'ent': 'Pacific'}],
 'hallucinations': ['killed at least 11 people and injured more than 100', "the Pacific nation's president has said."]
},
{'id': 9369,
 'pred': 'A Canadian man has described the moment he was struck by lightning during his son-in-law\'s wedding ceremony - just as he was about to tell him he was "a lucky guy".',
 'ents': [{'start': 2, 'end': 10, 'label': 0, 'type': 'NORP', 'ent': 'Canadian'}],
 'hallucinations': []
},
{'id': 9185,
 'pred': 'A baby pine marten has been captured on camera for the first time in Wales as part of a campaign to reintroduce the animal to Ceredigion.',
 'ents': [{'start': 55,
   'end': 60,
   'label': 0,
   'type': 'ORDINAL',
   'ent': 'first'},
  {'start': 69, 'end': 74, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': []},
{'id': 3611,
 'pred': "A Rwandan man who called for the mass murder of Tutsi people during the country's genocide has been sentenced to life in prison by a court in Rwanda.",
 'ents': [{'start': 2, 'end': 9, 'label': 0, 'type': 'ORG', 'ent': 'Rwandan'},
          {'start': 48, 'end': 53, 'label': 0, 'type': 'NORP', 'ent': 'Tutsi'},
          {'start': 142, 'end': 148, 'label': 0, 'type': 'GPE', 'ent': 'Rwanda'}],
 'hallucinations': []
},
{'id': 1754,
 'pred': 'Prince William has said he takes his duties as a monarch "very seriously" but does not take them "completely to heart" as he prepares to take on more responsibility.',
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'PERSON', 'ent': 'Prince'},
          {'start': 7, 'end': 14, 'label': 0, 'type': 'PERSON', 'ent': 'William'}],
 'hallucinations': []
},
{'id': 10244,
 'pred': 'A man has been arrested in China for allegedly driving a Lamborghini at more than twice the legal speed limit around Beijing, state media report.',
 'ents': [{'start': 27, 'end': 32, 'label': 0, 'type': 'GPE', 'ent': 'China'},
          {'start': 57, 'end': 68, 'label': 3, 'type': 'ORG', 'ent': 'Lamborghini'},
          {'start': 117, 'end': 124, 'label': 0, 'type': 'GPE', 'ent': 'Beijing'}],
 'hallucinations': []
},
{'id': 4850,
 'pred': 'A man and woman found dead at their home in Ballycastle, County Antrim, may have been dead for weeks, police have said.',
 'ents': [{'start': 44,
   'end': 55,
   'label': 0,
   'type': 'GPE',
   'ent': 'Ballycastle'},
  {'start': 57, 'end': 70, 'label': 1, 'type': 'PERSON', 'ent': 'County Antrim'},
  {'start': 95, 'end': 100, 'label': 0, 'type': 'DATE', 'ent': 'weeks'}],
 'hallucinations': []},
{'id': 1575,
 'pred': 'Rules requiring schools to promote "fundamental British values", including the role of law, are "deeply ill-considered" and "political posturing", a teachers\' union has warned.',
 'ents': [{'start': 48, 'end': 55, 'label': 0, 'type': 'NORP', 'ent': 'British'}],
 'hallucinations': []
},
{'id': 5686,
 'pred': 'Politicians on both sides of the EU referendum will have to work hard to ensure that the public is not left disappointed by the outcome of the referendum.',
 'ents': [{'start': 33, 'end': 35, 'label': 0, 'type': 'ORG', 'ent': 'EU'}],
 'hallucinations': []
},
{'id': 9254,
 'pred': 'Two men have been found guilty of murdering a man who was shot dead after leaving a pub in Greater Manchester.',
 'ents': [{'start': 0, 'end': 3, 'label': 3, 'type': 'CARDINAL', 'ent': 'Two'},
          {'start': 91, 'end': 109, 'label': 1, 'type': 'GPE', 'ent': 'Greater Manchester'}],
 'hallucinations': []
},
{'id': 2759,
 'pred': 'A chameleon has been put down by RSPCA Cymru after it was found injured and abandoned in a Cardiff park.',
 'ents': [{'start': 33, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'RSPCA Cymru'},
          {'start': 91, 'end': 98, 'label': 1, 'type': 'ORG', 'ent': 'Cardiff'}],
 'hallucinations': []
},
{'id': 8953,
 'pred': 'The BBC is to create a new Ideas Service to bring the "best from Britain\'s great cultural institutions and thinkers" to everyone, as part of a major overhaul of the corporation.',
 'ents': [{'start': 4, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'BBC'},
          {'start': 27, 'end': 40, 'label': 0, 'type': 'ORG', 'ent': 'Ideas Service'},
          {'start': 65, 'end': 72, 'label': 0, 'type': 'GPE', 'ent': 'Britain'}],
 'hallucinations': []
},
{'id': 7619,
 'pred': 'An NHS trust has been ordered to pay £3.6m in damages to the family of a baby girl who suffered severe brain damage after she was born in a C-section.',
 'ents': [{'start': 3, 'end': 6, 'label': 0, 'type': 'ORG', 'ent': 'NHS'},
  {'start': 38, 'end': 41, 'label': 2, 'type': 'MONEY', 'ent': '3.6'}],
 'hallucinations': []},
{'id': 700,
 'pred': 'The war on drugs has been fought for 50 years in the United States, with hundreds of thousands of people in jail and billions of dollars spent by the US and other countries on the issue.',
 'ents': [{'start': 37,
   'end': 45,
   'label': 0,
   'type': 'DATE',
   'ent': '50 years'},
  {'start': 49,
   'end': 66,
   'label': 0,
   'type': 'GPE',
   'ent': 'the United States'},
  {'start': 73,
   'end': 94,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'hundreds of thousands'},
  {'start': 117,
   'end': 128,
   'label': 0,
   'type': 'MONEY',
   'ent': 'billions of'},
  {'start': 150, 'end': 152, 'label': 0, 'type': 'GPE', 'ent': 'US'}],
 'hallucinations': []},
{'id': 3379,
 'pred': 'Stagecoach has announced plans to invest more than £100m in new buses and coaches over the next three years.',
 'ents': [{'start': 0, 'end': 10, 'label': 0, 'type': 'ORG', 'ent': 'Stagecoach'},
          {'start': 41, 'end': 56, 'label': 2, 'type': 'CARDINAL', 'ent': 'more than £100m'},
          {'start': 87, 'end': 107, 'label': 2, 'type': 'DATE', 'ent': 'the next three years'}],
 'hallucinations': ['more than £100m', 'over the next three years']
},
{'id': 7021,
 'pred': 'A woman has been given a suspended prison sentence for defrauding a charity set up in memory of a Cumbrian teenager who died from cancer.',
 'ents': [{'start': 98, 'end': 106, 'label': 0, 'type': 'NORP', 'ent': 'Cumbrian'}],
 'hallucinations': []
},
{'id': 3180,
 'pred': 'Burton Albion goalkeeper Jon McLaughlin is one of eight players to be released by the Championship club following their relegation from the Championship.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Burton'},
  {'start': 7, 'end': 13, 'label': 1, 'type': 'PERSON', 'ent': 'Albion'},
  {'start': 25, 'end': 28, 'label': 1, 'type': 'PERSON', 'ent': 'Jon'},
  {'start': 29,
   'end': 39,
   'label': 0,
   'type': 'PERSON',
   'ent': 'McLaughlin'},
  {'start': 43, 'end': 46, 'label': 0, 'type': 'CARDINAL', 'ent': 'one'},
  {'start': 50, 'end': 55, 'label': 2, 'type': 'CARDINAL', 'ent': 'eight'}],
 'hallucinations': []},
{'id': 3030,
 'pred': 'Former England midfielder David Ginola has launched his bid to succeed Sepp Blatter as Fifa president, saying the current system is "not working".',
 'ents': [{'start': 7,
   'end': 14,
   'label': 2,
   'type': 'PERSON',
   'ent': 'England'},
  {'start': 26, 'end': 31, 'label': 0, 'type': 'PERSON', 'ent': 'David'},
  {'start': 32, 'end': 38, 'label': 0, 'type': 'PERSON', 'ent': 'Ginola'},
  {'start': 71, 'end': 75, 'label': 1, 'type': 'PERSON', 'ent': 'Sepp'},
  {'start': 76, 'end': 83, 'label': 0, 'type': 'PERSON', 'ent': 'Blatter'},
  {'start': 87, 'end': 91, 'label': 0, 'type': 'ORG', 'ent': 'Fifa'}],
 'hallucinations': []},
{'id': 191,
 'pred': 'Work to improve a roundabout on the A55 and A483 in Chester has been delayed for a second time, causing "chaos" for motorists.',
 'ents': [{'start': 36, 'end': 39, 'label': 0, 'type': 'ORG', 'ent': 'A55'},
          {'start': 44, 'end': 48, 'label': 0, 'type': 'ORG', 'ent': 'A483'},
          {'start': 52, 'end': 59, 'label': 0, 'type': 'GPE', 'ent': 'Chester'},
          {'start': 83, 'end': 89, 'label': 3, 'type': 'ORDINAL', 'ent': 'second'}],
 'hallucinations': ['for a second time']
},
{'id': 8770,
 'pred': 'Oxford United midfielder Callum Kenny has signed a new one-year contract with the League One club, keeping him at the club until the summer of 2018.',
 'ents': [{'start': 0, 'end': 13, 'label': 1, 'type': 'ORG', 'ent': 'Oxford United'},
          {'start': 25, 'end': 31, 'label': 2, 'type': 'PERSON', 'ent': 'Callum'},
          {'start': 32, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'Kenny'},
          {'start': 55, 'end': 63, 'label': 2, 'type': 'DATE', 'ent': 'one-year'},
          {'start': 82, 'end': 92, 'label': 2, 'type': 'ORG', 'ent': 'League One'},
          {'start': 129, 'end': 147, 'label': 2, 'type': 'DATE', 'ent': 'the summer of 2018'}],
 'hallucinations': ['Callum', 'one-year contract', 'the League One club', 'the summer of 2018']},
{'id': 5620,
 'pred': 'A second sinkhole has opened up in Manchester city centre, the latest in a series of sinkholes to open up in the city.',
 'ents': [{'start': 2, 'end': 8, 'label': 3, 'type': 'ORDINAL', 'ent': 'second'},
          {'start': 35, 'end': 45, 'label': 0, 'type': 'GPE', 'ent': 'Manchester'}],
 'hallucinations': ['second']
},
{'id': 1948,
 'pred': 'A mother and her partner have appeared in court charged in connection with the death of a six-year-old boy who was found dead at a water park.',
 'ents': [{'start': 90, 'end': 102, 'label': 2, 'type': 'DATE', 'ent': 'six-year-old'}],
 'hallucinations': ['six-year-old']
},
{'id': 10,
 'pred': 'A hospital where a patient died and another was left sitting on a bedpan for more than an hour in the emergency department has been ordered to improve its A&E services after inspectors found poor standards of care.',
 'ents': [{'start': 77, 'end': 94, 'label': 0, 'type': 'TIME', 'ent': 'more than an hour'},
          {'start': 155, 'end': 158, 'label': 0, 'type': 'ORG', 'ent': 'A&E'}],
 'hallucinations': []
},
{'id': 2156,
 'pred': "The public has been asked to vote for their favourite bird to be named as the UK's national bird ahead of the general election.",
 'ents': [{'start': 78, 'end': 80, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []
},
{'id': 2991,
 'pred': "A man who hid his girlfriend's body in a cupboard and took her benefits for several months after her death has been jailed.",
 'ents': [{'start': 76, 'end': 90, 'label': 0, 'type': 'DATE', 'ent': 'several months'}],
 'hallucinations': []
},
{'id': 8702,
 'pred': 'Eritrea has accused the United Nations of carrying out an "unprecedented attack" on its government after it published a report accusing the Eritrean government of human rights abuses.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'GPE', 'ent': 'Eritrea'},
          {'start': 20, 'end': 38, 'label': 0, 'type': 'ORG', 'ent': 'the United Nations'},
          {'start': 140, 'end': 148, 'label': 0, 'type': 'NORP', 'ent': 'Eritrean'}],
 'hallucinations': ['"unprecedented attack"']
},
{'id': 3466,
 'pred': 'The body of a man has been found off the coast of St Fergus in Aberdeenshire, police have confirmed.',
 'ents': [{'start': 50, 'end': 59, 'label': 0, 'type': 'GPE', 'ent': 'St Fergus'},
          {'start': 63, 'end': 76, 'label': 1, 'type': 'GPE', 'ent': 'Aberdeenshire'}],
 'hallucinations': ['in Aberdeenshire']
},
{'id': 3088,
 'pred': "Adele's new single, Hello, has become the fastest-selling single of the year in the UK, with more than 200,000 downloads in its first 24 hours.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'PERSON', 'ent': 'Adele'},
          {'start': 84, 'end': 86, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 93, 'end': 110, 'label': 1, 'type': 'CARDINAL', 'ent': 'more than 200,000'},
          {'start': 128, 'end': 142, 'label': 2, 'type': 'TIME', 'ent': 'first 24 hours'}],
 'hallucinations': []
},
{'id': 5706,
 'pred': "Kenya's main opposition leader, Raila Odinga, has called for a one-day strike on Monday to protest against the result of last week's presidential election.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'GPE', 'ent': 'Kenya'},
          {'start': 32, 'end': 37, 'label': 1, 'type': 'PERSON', 'ent': 'Raila'},
          {'start': 38, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'Odinga'},
          {'start': 63, 'end': 70, 'label': 2, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 81, 'end': 87, 'label': 2, 'type': 'DATE', 'ent': 'Monday'},
          {'start': 121, 'end': 132, 'label': 0, 'type': 'DATE', 'ent': "last week's"}],
 'hallucinations': ['Raila', 'one-day strike on Monday']
},
{'id': 5290,
 'pred': 'A hacking group based in North Korea is believed to have been behind the global ransomware attack that hit the NHS in May, the BBC has learned.',
 'ents': [{'start': 25, 'end': 36, 'label': 0, 'type': 'GPE', 'ent': 'North Korea'},
          {'start': 111, 'end': 114, 'label': 0, 'type': 'ORG', 'ent': 'NHS'},
          {'start': 118, 'end': 121, 'label': 0, 'type': 'DATE', 'ent': 'May'},
          {'start': 127, 'end': 130, 'label': 0, 'type': 'ORG', 'ent': 'BBC'}],
 'hallucinations': []
},
{'id': 7871,
 'pred': "A man in his 20s has been arrested on suspicion of attempted murder after a man was stabbed outside a McDonald's in Londonderry.",
 'ents': [{'start': 13, 'end': 16, 'label': 0, 'type': 'DATE', 'ent': '20s'},
          {'start': 102, 'end': 110, 'label': 1, 'type': 'ORG', 'ent': 'McDonald'},
          {'start': 116, 'end': 127, 'label': 2, 'type': 'GPE', 'ent': 'Londonderry'}],
 'hallucinations': ['has been arrested', "outside a McDonald's in Londonderry."]
},
{'id': 6041,
 'pred': 'A ride at Alton Towers is to reopen to the public next month, six months after a rollercoaster crash that left five people injured.',
 'ents': [{'start': 10,
   'end': 22,
   'label': 1,
   'type': 'ORG',
   'ent': 'Alton Towers'},
  {'start': 50,
   'end': 60,
   'label': 2,
   'type': 'DATE',
   'ent': 'next month'},
  {'start': 62, 'end': 72, 'label': 2, 'type': 'DATE', 'ent': 'six months'},
  {'start': 111,
   'end': 115,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'five'}],
 'hallucinations': []},
{'id': 1110,
 'pred': 'Chelsea manager Jose Mourinho said the Premier League is "not the best league in the world" after his side\'s 1-0 defeat by West Ham was like "football from the 19th Century".',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Chelsea'},
          {'start': 16, 'end': 20, 'label': 0, 'type': 'PERSON', 'ent': 'Jose'},
          {'start': 21, 'end': 29, 'label': 0, 'type': 'PERSON', 'ent': 'Mourinho'},
          {'start': 109, 'end': 112, 'label': 2, 'type': 'CARDINAL', 'ent': '1-0'},
          {'start': 123, 'end': 131, 'label': 0, 'type': 'LOC', 'ent': 'West Ham'},
          {'start': 156, 'end': 172, 'label': 0, 'type': 'DATE', 'ent': 'the 19th Century'}],
 'hallucinations': ['1-0 defeat by West Ham']
},
{'id': 8242,
 'pred': "Nigeria's main opposition party has suffered its biggest defeat since the end of military rule, in elections for state governors and state assemblies.",
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'GPE',
   'ent': 'Nigeria'}],
 'hallucinations': []},
{'id': 9526,
 'pred': "The first World Cup was held in India in 1950, and the country's first football team, the Indian Football Federation (IFF), won a place in the final of the inaugural tournament.",
 'ents': [{'start': 4, 'end': 9, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 10, 'end': 19, 'label': 0, 'type': 'EVENT', 'ent': 'World Cup'},
          {'start': 32, 'end': 37, 'label': 0, 'type': 'GPE', 'ent': 'India'},
          {'start': 41, 'end': 45, 'label': 1, 'type': 'DATE', 'ent': '1950'},
          {'start': 65, 'end': 70, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 86, 'end': 116, 'label': 2, 'type': 'ORG', 'ent': 'the Indian Football Federation'}],
 'hallucinations': ['The first World Cup was held in India in 1950', 'Indian Football Federation (IFF)']
},
{'id': 9340,
 'pred': 'Cardiff Blues scrum-half Mike Phillips has been suspended from the Wales squad after being involved in an incident outside a restaurant in Cardiff.',
 'ents': [{'start': 0, 'end': 13, 'label': 0, 'type': 'ORG', 'ent': 'Cardiff Blues'},
          {'start': 25, 'end': 29, 'label': 0, 'type': 'PERSON', 'ent': 'Mike'},
          {'start': 30, 'end': 38, 'label': 0, 'type': 'PERSON', 'ent': 'Phillips'},
          {'start': 67, 'end': 72, 'label': 0, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 139, 'end': 146, 'label': 0, 'type': 'GPE', 'ent': 'Cardiff'}],
 'hallucinations': []},
{'id': 11046,
 'pred': 'A man who led police on a high-speed chase for more than 50 minutes after he was caught driving without a licence has been jailed.',
 'ents': [{'start': 57, 'end': 67, 'label': 0, 'type': 'TIME', 'ent': '50 minutes'}],
 'hallucinations': []},
{'id': 4947, # NOTE
 'pred': 'Universities in England and Wales are to stage a one-day strike on Wednesday in a dispute over pay, in a row over the size of a pay rise.',
 'ents': [{'start': 16, 'end': 23, 'label': 1, 'type': 'GPE', 'ent': 'England'},
          {'start': 28, 'end': 33, 'label': 1, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 49, 'end': 56, 'label': 2, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 67, 'end': 76, 'label': 3, 'type': 'DATE', 'ent': 'Wednesday'}],
 'hallucinations': []},
{'id': 3858,
 'pred': 'Gardeners in Northern Ireland are being urged to get rid of a "thug" plant that is out-competing bluebells in their gardens.',
 'ents': [{'start': 13, 'end': 29, 'label': 2, 'type': 'GPE', 'ent': 'Northern Ireland'}],
 'hallucinations': ['in Northern Ireland']},
{'id': 1843, # how is this possbile
 'pred': 'A five-year-old boy with a rare form of cancer who was cheered on by Sunderland fans when he appeared as a mascot for the team has lost his fight against the disease.',
 'ents': [{'start': 2, 'end': 15, 'label': 1, 'type': 'DATE', 'ent': 'five-year-old'},
          {'start': 69, 'end': 79, 'label': 0, 'type': 'ORG', 'ent': 'Sunderland'}],
 'hallucinations': []},
{'id': 4867,
 'pred': 'Work permits for EU citizens living and working in the UK could cut net migration by up to 100,000 a year if the UK voted to leave the EU, according to a report from a think tank.',
 'ents': [{'start': 17, 'end': 19, 'label': 0, 'type': 'ORG', 'ent': 'EU'},
          {'start': 55, 'end': 57, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 91, 'end': 98, 'label': 0, 'type': 'CARDINAL', 'ent': '100,000'},
          {'start': 113, 'end': 115, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 135, 'end': 137, 'label': 0, 'type': 'ORG', 'ent': 'EU'}],
 'hallucinations': []},
{'id': 1472,
 'pred': 'A man armed with a knife has robbed a van driver in Edinburgh after he dropped off a man he had picked up in the city centre.',
 'ents': [{'start': 52, 'end': 61, 'label': 1, 'type': 'GPE', 'ent': 'Edinburgh'}],
 'hallucinations': []},
{'id': 10112,
 'pred': 'Three men and two teenagers have been arrested following disturbances at the Celtic v Aberdeen match at Pittodrie on Friday night.',
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'CARDINAL', 'ent': 'Three'},
          {'start': 14, 'end': 17, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
          {'start': 77, 'end': 83, 'label': 0, 'type': 'NORP', 'ent': 'Celtic'},
          {'start': 86, 'end': 94, 'label': 0, 'type': 'ORG', 'ent': 'Aberdeen'},
          {'start': 104, 'end': 113, 'label': 1, 'type': 'ORG', 'ent': 'Pittodrie'},
          {'start': 117, 'end': 123, 'label': 0, 'type': 'DATE', 'ent': 'Friday'}],
 'hallucinations': []},
{'id': 9203,
 'pred': 'More than 100 drivers a day have been caught speeding on a stretch of the M4 in Neath Port Talbot since speed cameras went into operation, figures have shown.',
 'ents': [{'start': 80, 'end': 97, 'label': 1, 'type': 'ORG', 'ent': 'Neath Port Talbot'}],
 'hallucinations': []},
{'id': 9166,
 'pred': 'Port Vale came from behind to beat West Bromwich Albion and reach the second round of the League One play-off semi-finals.',
 'ents': [{'start': 0, 'end': 9, 'label': 1, 'type': 'ORG', 'ent': 'Port Vale'},
          {'start': 35, 'end': 55, 'label': 1, 'type': 'ORG', 'ent': 'West Bromwich Albion'},
          {'start': 70, 'end': 76, 'label': 2, 'type': 'ORDINAL', 'ent': 'second'}],
 'hallucinations': ["the second round of the League One play-off semi-finals."]},
{'id': 2333,
 'pred': "Donald Trump's decision to pull the US out of the Paris Agreement on climate change was a blow to Rex Tillerson.",
 'ents': [{'start': 0, 'end': 6, 'label': 1, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 7, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': "Trump"},
          {'start': 36, 'end': 38, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 46, 'end': 65, 'label': 0, 'type': 'ORG', 'ent': 'the Paris Agreement'},
          {'start': 98, 'end': 101, 'label': 1, 'type': 'PERSON', 'ent': 'Rex'},
          {'start': 102, 'end': 111, 'label': 0, 'type': 'PERSON', 'ent': 'Tillerson'}],
 'hallucinations': []},
{'id': 10898,
 'pred': 'Amnesty International has called on the Indian government to repeal a controversial law that gives security forces sweeping powers in Indian-administered Kashmir, saying it has contributed to human rights abuses.',
 'ents': [{'start': 0, 'end': 21, 'label': 0, 'type': 'ORG', 'ent': 'Amnesty International'},
          {'start': 40, 'end': 46, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
          {'start': 134, 'end': 140, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
          {'start': 154, 'end': 161, 'label': 0, 'type': 'LOC', 'ent': 'Kashmir'}],
 'hallucinations': []},
{'id': 8921,
 'pred': 'The UK\'s Department for Environment, Food and Rural Affairs (Defra) has called for "more realistic field trials" of two common pesticides after research suggested they could be damaging to honey bees.',
 'ents': [{'start': 4, 'end': 6, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 9, 'end': 59, 'label': 1, 'type': 'ORG', 'ent': 'Department for Environment, Food and Rural Affairs'},
          {'start': 116, 'end': 119, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'}],
 'hallucinations': []},
{'id': 572,
 'pred': 'A police investigation has been launched into the disappearance of a car that has been missing from a Hertfordshire garage for almost 20 years.',
 'ents': [{'start': 134, 'end': 142, 'label': 0, 'type': 'DATE', 'ent': '20 years'}],
 'hallucinations': []},
{'id': 5113,
 'pred': 'A volcano in Costa Rica has erupted for the first time in six years, spewing huge plumes of ash and gas into the sky over the capital and surrounding areas.',
 'ents': [{'start': 13, 'end': 23, 'label': 0, 'type': 'GPE', 'ent': 'Costa Rica'},
          {'start': 44, 'end': 49, 'label': 2, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 58, 'end': 67, 'label': 0, 'type': 'DATE', 'ent': 'six years'}],
 'hallucinations': ['for the first time']},
{'id': 6920,
 'pred': 'A Dundee United fan has admitted striking a Rangers fan in the head with a metal pole at the end of the Scottish Premiership match between the sides.',
 'ents': [{'start': 2, 'end': 15, 'label': 0, 'type': 'ORG', 'ent': 'Dundee United'},
          {'start': 44, 'end': 51, 'label': 0, 'type': 'ORG', 'ent': 'Rangers'},
          {'start': 104, 'end': 124, 'label': 2, 'type': 'NORP', 'ent': 'Scottish Premiership'}],
 'hallucinations': []},
{'id': 3590,
 'pred': 'A court in New York has ordered the trial of Hank Greenberg, the former co-founder of US insurance giant AIG, to begin next month on charges of fraud.',
 'ents': [{'start': 11,
   'end': 19,
   'label': 0,
   'type': 'GPE',
   'ent': 'New York'},
  {'start': 45, 'end': 49, 'label': 0, 'type': 'PERSON', 'ent': 'Hank'},
  {'start': 50,
   'end': 59,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Greenberg'},
  {'start': 86, 'end': 88, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 105, 'end': 108, 'label': 1, 'type': 'PRODUCT', 'ent': 'AIG'},
  {'start': 119,
   'end': 129,
   'label': 2,
   'type': 'DATE',
   'ent': 'next month'}],
 'hallucinations': []},
{'id': 3278,
 'pred': "French centrist presidential candidate Emmanuel Macron has accepted an offer to form an alliance with a former prime minister to fight far-right candidate Marine Le Pen in next month's election.",
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'NORP', 'ent': 'French'},
          {'start': 39, 'end': 47, 'label': 1, 'type': 'PERSON', 'ent': 'Emmanuel'},
          {'start': 48, 'end': 54, 'label': 0, 'type': 'PERSON', 'ent': 'Macron'},
          {'start': 155, 'end': 161, 'label': 0, 'type': 'PERSON', 'ent': 'Marine'},
          {'start': 162, 'end': 164, 'label': 0, 'type': 'PERSON', 'ent': 'Le'},
          {'start': 165, 'end': 168, 'label': 0, 'type': 'PERSON', 'ent': 'Pen'},
          {'start': 172, 'end': 184, 'label': 2, 'type': 'DATE', 'ent': "next month's"}],
 'hallucinations': []},
{'id': 6117,
 'pred': 'A Porto-based football player has been banned for four years for attacking a referee and three months for assaulting an opposition player in an amateur league match in the Portuguese city.',
 'ents': [{'start': 2,
   'end': 7,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Porto'},
  {'start': 50, 'end': 60, 'label': 0, 'type': 'DATE', 'ent': 'four years'},
  {'start': 89,
   'end': 101,
   'label': 0,
   'type': 'DATE',
   'ent': 'three months'},
  {'start': 172,
   'end': 182,
   'label': 1,
   'type': 'NORP',
   'ent': 'Portuguese'}],
 'hallucinations': []},
{'id': 7688,
 'pred': 'Sam Baldock scored the only goal of the game as Brighton beat Wolves to move up to third in the Championship with victory at the Amex Stadium.',
 'ents': [{'start': 0,
   'end': 3,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Sam'},
  {'start': 4, 'end': 11, 'label': 0, 'type': 'PERSON', 'ent': 'Baldock'},
  {'start': 83, 'end': 88, 'label': 2, 'type': 'ORDINAL', 'ent': 'third'},
  {'start': 125,
   'end': 141,
   'label': 2,
   'type': 'FAC',
   'ent': 'the Amex Stadium'}],
 'hallucinations': []},
{'id': 9758,
 'pred': 'Stoke City defender Ryan Shawcross has been ruled out for up to four weeks with a hamstring injury and could miss the rest of the season.',
 'ents': [{'start': 20,
   'end': 24,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Ryan'},
  {'start': 25,
   'end': 34,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Shawcross'},
  {'start': 64,
   'end': 74,
   'label': 0,
   'type': 'DATE',
   'ent': 'four weeks'}],
 'hallucinations': []},
{'id': 1733,
 'pred': "A man has been found guilty of the manslaughter of his mother's partner who died after a fight at a Dorset holiday park.",
 'ents': [{'start': 100,
   'end': 106,
   'label': 1,
   'type': 'GPE',
   'ent': 'Dorset'}],
 'hallucinations': []},
{'id': 3432,
 'pred': 'A search is under way to find a mystery photographer who took the last photograph of a woman before she died, after she was found dead in a garden at a Kent castle.',
 'ents': [{'start': 152,
   'end': 156,
   'label': 3,
   'type': 'PERSON',
   'ent': 'Kent'}],
 'hallucinations': []},
{'id': 1264,
 'pred': 'Former Scotland boss Alex McLeish has agreed a deal to become the new coach of Egyptian club Zamalek, the club have confirmed.',
 'ents': [{'start': 7,
   'end': 15,
   'label': 0,
   'type': 'GPE',
   'ent': 'Scotland'},
  {'start': 21, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Alex'},
  {'start': 26, 'end': 33, 'label': 0, 'type': 'PERSON', 'ent': 'McLeish'},
  {'start': 79, 'end': 87, 'label': 0, 'type': 'NORP', 'ent': 'Egyptian'},
  {'start': 93,
   'end': 100,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Zamalek'}],
 'hallucinations': []},
{'id': 5195,
 'pred': 'A Church of Ireland bishop has said the church is "not up to pace" with the debate on same-sex marriage in Northern Ireland.',
 'ents': [{'start': 2,
   'end': 19,
   'label': 0,
   'type': 'ORG',
   'ent': 'Church of Ireland'},
  {'start': 107,
   'end': 123,
   'label': 0,
   'type': 'GPE',
   'ent': 'Northern Ireland'}],
 'hallucinations': []},
{'id': 1436,
 'pred': 'Labour leader Jeremy Corbyn has said he would allow his MPs a "free vote" on whether the UK should take part in air strikes against Islamic State militants in Syria.',
 'ents': [{'start': 14,
   'end': 20,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Jeremy'},
  {'start': 21, 'end': 27, 'label': 0, 'type': 'PERSON', 'ent': 'Corbyn'},
  {'start': 89, 'end': 91, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
  {'start': 132, 'end': 139, 'label': 0, 'type': 'NORP', 'ent': 'Islamic'},
  {'start': 159, 'end': 164, 'label': 0, 'type': 'GPE', 'ent': 'Syria'}],
 'hallucinations': []},
{'id': 3210,
 'pred': 'Liverpool manager Jurgen Klopp says Steven Gerrard has a "bright future" as a manager after the LA Galaxy legend announced he is to return to Anfield.',
 'ents': [{'start': 0,
   'end': 9,
   'label': 0,
   'type': 'ORG',
   'ent': 'Liverpool'},
  {'start': 18, 'end': 24, 'label': 1, 'type': 'PERSON', 'ent': 'Jurgen'},
  {'start': 25, 'end': 30, 'label': 0, 'type': 'PERSON', 'ent': 'Klopp'},
  {'start': 36, 'end': 42, 'label': 0, 'type': 'PERSON', 'ent': 'Steven'},
  {'start': 43, 'end': 50, 'label': 0, 'type': 'PERSON', 'ent': 'Gerrard'},
  {'start': 96, 'end': 105, 'label': 0, 'type': 'ORG', 'ent': 'LA Galaxy'},
  {'start': 142, 'end': 149, 'label': 0, 'type': 'ORG', 'ent': 'Anfield'}],
 'hallucinations': []},
{'id': 6066,
 'pred': "England head coach Eddie Jones has defended the targeting of Ireland fly-half Johnny Sexton in Saturday's Six Nations match against England, saying it is part of the game.",
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'GPE',
   'ent': 'England'},
  {'start': 19, 'end': 24, 'label': 1, 'type': 'PERSON', 'ent': 'Eddie'},
  {'start': 25, 'end': 30, 'label': 0, 'type': 'PERSON', 'ent': 'Jones'},
  {'start': 61, 'end': 68, 'label': 0, 'type': 'GPE', 'ent': 'Ireland'},
  {'start': 78, 'end': 84, 'label': 1, 'type': 'PERSON', 'ent': 'Johnny'},
  {'start': 85, 'end': 91, 'label': 0, 'type': 'PERSON', 'ent': 'Sexton'},
  {'start': 95, 'end': 103, 'label': 1, 'type': 'DATE', 'ent': 'Saturday'},
  {'start': 106, 'end': 109, 'label': 1, 'type': 'CARDINAL', 'ent': 'Six'},
  {'start': 132, 'end': 139, 'label': 0, 'type': 'GPE', 'ent': 'England'}],
 'hallucinations': []},
{'id': 4186,
 'pred': 'The UK and France are holding talks in Calais to try to reach an agreement on how to deal with the migrant crisis in the port city.',
 'ents': [{'start': 4, 'end': 6, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
  {'start': 11, 'end': 17, 'label': 0, 'type': 'GPE', 'ent': 'France'},
  {'start': 39, 'end': 45, 'label': 0, 'type': 'GPE', 'ent': 'Calais'}],
 'hallucinations': []},
{'id': 8646,
 'pred': 'Birmingham City midfielder James Morrison says manager Gary Rowett was right to criticise his side\'s "lack of desire" after they were held to a draw by Barnsley.',
 'ents': [{'start': 0,
   'end': 15,
   'label': 1,
   'type': 'GPE',
   'ent': 'Birmingham City'},
  {'start': 27, 'end': 32, 'label': 2, 'type': 'PERSON', 'ent': 'James'},
  {'start': 33, 'end': 41, 'label': 0, 'type': 'PERSON', 'ent': 'Morrison'},
  {'start': 55, 'end': 59, 'label': 1, 'type': 'PERSON', 'ent': 'Gary'},
  {'start': 60, 'end': 66, 'label': 0, 'type': 'PERSON', 'ent': 'Rowett'},
  {'start': 152,
   'end': 160,
   'label': 2,
   'type': 'LOC',
   'ent': 'Barnsley'}],
 'hallucinations': []},
{'id': 6554,
 'pred': 'Leicester Tigers have extended the loan spell of England back-row Ellis Genge from Bristol until the end of the season.',
 'ents': [{'start': 49,
   'end': 56,
   'label': 0,
   'type': 'GPE',
   'ent': 'England'},
  {'start': 66, 'end': 71, 'label': 0, 'type': 'PERSON', 'ent': 'Ellis'},
  {'start': 72, 'end': 77, 'label': 0, 'type': 'PERSON', 'ent': 'Genge'},
  {'start': 83, 'end': 90, 'label': 0, 'type': 'PERSON', 'ent': 'Bristol'},
  {'start': 97,
   'end': 118,
   'label': 2,
   'type': 'DATE',
   'ent': 'the end of the season'}],
 'hallucinations': []},
{'id': 6054,
 'pred': 'Transformers: Age of Extinction has topped the US box office for the fourth weekend in a row, despite the absence of any major movies to coincide with the Independence Day holiday.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'GPE', 'ent': 'Transformers'},
          {'start': 14, 'end': 31, 'label': 1, 'type': 'GPE', 'ent': 'Age of Extinction'},
          {'start': 47, 'end': 49, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 65, 'end': 92, 'label': 2, 'type': 'DATE', 'ent': 'the fourth weekend in a row'},
          {'start': 151, 'end': 171, 'label': 0, 'type': 'DATE', 'ent': 'the Independence Day'}],
 'hallucinations': []},
{'id': 11143,
 'pred': 'Derby County captain Andy Keogh has signed a new three-year contract with the Championship club.',
 'ents': [{'start': 0,
   'end': 12,
   'label': 1,
   'type': 'GPE',
   'ent': 'Derby County'},
  {'start': 21, 'end': 25, 'label': 2, 'type': 'PERSON', 'ent': 'Andy'},
  {'start': 26, 'end': 31, 'label': 0, 'type': 'PERSON', 'ent': 'Keogh'},
  {'start': 49,
   'end': 59,
   'label': 1,
   'type': 'DATE',
   'ent': 'three-year'}],
 'hallucinations': []},
{'id': 1147,
 'pred': 'A World War Two fighter jet used in the Gulf War has been put up for sale by two ex-RAF engineers who have spent more than 20 years restoring it.',
 'ents': [{'start': 2,
   'end': 11,
   'label': 0,
   'type': 'EVENT',
   'ent': 'World War'},
  {'start': 12, 'end': 15, 'label': 0, 'type': 'CARDINAL', 'ent': 'Two'},
  {'start': 36,
   'end': 48,
   'label': 0,
   'type': 'EVENT',
   'ent': 'the Gulf War'},
  {'start': 77, 'end': 80, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
  {'start': 81, 'end': 87, 'label': 0, 'type': 'ORG', 'ent': 'ex-RAF'},
  {'start': 113,
   'end': 131,
   'label': 2,
   'type': 'DATE',
   'ent': 'more than 20 years'}],
 'hallucinations': []},
{'id': 9629,
 'pred': "Teachers' unions in Northern Ireland have rejected a pay offer from the Department of Education and their employers in a long-running dispute over pay.",
 'ents': [{'start': 20,
   'end': 36,
   'label': 0,
   'type': 'GPE',
   'ent': 'Northern Ireland'},
  {'start': 68,
   'end': 95,
   'label': 0,
   'type': 'ORG',
   'ent': 'the Department of Education'}],
 'hallucinations': []},
{'id': 621,
 'pred': 'Former NBA basketball star Lamar Odom has been taken to hospital after being found unconscious in a Nevada brothel, police say.',
 'ents': [{'start': 7, 'end': 10, 'label': 0, 'type': 'ORG', 'ent': 'NBA'},
  {'start': 27, 'end': 32, 'label': 0, 'type': 'PERSON', 'ent': 'Lamar'},
  {'start': 33, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'Odom'},
  {'start': 100, 'end': 106, 'label': 0, 'type': 'GPE', 'ent': 'Nevada'}],
 'hallucinations': []},
{'id': 10349,
 'pred': 'Scottish Attorney General Michael Matheson has said it would be "better for all parties" to refrain from making comment while the investigation into the death of Kelvin Bayoh is taking place.',
 'ents': [{'start': 0,
   'end': 8,
   'label': 1,
   'type': 'NORP',
   'ent': 'Scottish'},
  {'start': 9, 'end': 25, 'label': 2, 'type': 'NORP', 'ent': 'Attorney General'},
  {'start': 26, 'end': 33, 'label': 1, 'type': 'PERSON', 'ent': 'Michael'},
  {'start': 34, 'end': 42, 'label': 0, 'type': 'PERSON', 'ent': 'Matheson'},
  {'start': 162, 'end': 168, 'label': 2, 'type': 'PERSON', 'ent': 'Kelvin'},
  {'start': 169, 'end': 174, 'label': 0, 'type': 'PERSON', 'ent': 'Bayoh'}],
 'hallucinations': []},
{'id': 730,
 'pred': "Conservative MP Zac Goldsmith has been chosen as the party's candidate for London mayor after winning a ballot of his constituents.",
 'ents': [{'start': 16,
   'end': 19,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Zac'},
  {'start': 20,
   'end': 29,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Goldsmith'},
  {'start': 75, 'end': 81, 'label': 0, 'type': 'GPE', 'ent': 'London'}],
 'hallucinations': []},
{'id': 124,
 'pred': "Wales have recalled midfielders Jonny Chester, Chris Dummett and Chris Matthews for Saturday's Euro 2016 qualifier against Belgium in Cardiff.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'GPE', 'ent': 'Wales'},
  {'start': 32, 'end': 37, 'label': 2, 'type': 'PERSON', 'ent': 'Jonny'},
  {'start': 38, 'end': 45, 'label': 0, 'type': 'PERSON', 'ent': 'Chester'},
  {'start': 47, 'end': 52, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
  {'start': 53, 'end': 60, 'label': 0, 'type': 'PERSON', 'ent': 'Dummett'},
  {'start': 65, 'end': 70, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
  {'start': 71, 'end': 79, 'label': 0, 'type': 'PERSON', 'ent': 'Matthews'},
  {'start': 84, 'end': 92, 'label': 2, 'type': 'DATE', 'ent': 'Saturday'},
  {'start': 95, 'end': 99, 'label': 0, 'type': 'EVENT', 'ent': 'Euro'},
  {'start': 123, 'end': 130, 'label': 0, 'type': 'GPE', 'ent': 'Belgium'},
  {'start': 134, 'end': 141, 'label': 0, 'type': 'DATE', 'ent': 'Cardiff'}],
 'hallucinations': []},
{'id': 5490,
 'pred': 'A nurse who secretly filmed himself raping women who were in hospital and volunteering at a music festival has been jailed for 12 years.',
 'ents': [{'start': 127,
   'end': 135,
   'label': 2,
   'type': 'DATE',
   'ent': '12 years'}],
 'hallucinations': []},
{'id': 5993,
 'pred': 'The Burmese government has set up a commission to investigate the violence that erupted in the western state of Rakhine in June, killing more than 50 people.',
 'ents': [{'start': 4,
   'end': 11,
   'label': 0,
   'type': 'NORP',
   'ent': 'Burmese'},
  {'start': 112,
   'end': 119,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Rakhine'},
  {'start': 123, 'end': 127, 'label': 3, 'type': 'DATE', 'ent': 'June'},
  {'start': 137,
   'end': 149,
   'label': 2,
   'type': 'CARDINAL',
   'ent': 'more than 50'}],
 'hallucinations': []},
{'id': 4004,
 'pred': "The world's tallest obelisk is being surveyed for the first time in more than a decade as part of a £1.5m project to help improve its safety.",
 'ents': [{'start': 101, 'end': 104, 'label': 2, 'type': 'MONEY', 'ent': '1.5'}],
 'hallucinations': []},
{'id': 11022,
 'pred': 'Edinburgh is gearing up for its annual three-week arts festival, which begins on Sunday with the opening of the International Festival.',
 'ents': [{'start': 0,
   'end': 9,
   'label': 0,
   'type': 'ORG',
   'ent': 'Edinburgh'},
  {'start': 39, 'end': 49, 'label': 0, 'type': 'DATE', 'ent': 'three-week'},
  {'start': 81, 'end': 87, 'label': 0, 'type': 'DATE', 'ent': 'Sunday'},
  {'start': 108,
   'end': 134,
   'label': 0,
   'type': 'EVENT',
   'ent': 'the International Festival'}],
 'hallucinations': []},
{'id': 6934,
 'pred': 'A 19-year-old man has been shot in the leg in a paramilitary-style attack in west Belfast.',
 'ents': [{'start': 77,
   'end': 89,
   'label': 2,
   'type': 'LOC',
   'ent': 'west Belfast'}],
 'hallucinations': []},
{'id': 3952,
 'pred': 'The family of a man who lost two teeth after he was punched in the face by a nurse at a Gloucestershire residential hospital have said they are "devastated" by what they believe was a cover-up.',
 'ents': [{'start': 29,
   'end': 32,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'two'},
  {'start': 88,
   'end': 103,
   'label': 3,
   'type': 'GPE',
   'ent': 'Gloucestershire'}],
 'hallucinations': []},
{'id': 790,
 'pred': 'Scientists have released new images of an asteroid called 2004 BL86, which has passed by the Earth for the first time.',
 'ents': [{'start': 58,
   'end': 62,
   'label': 0,
   'type': 'DATE',
   'ent': '2004'},
  {'start': 63, 'end': 67, 'label': 0, 'type': 'PRODUCT', 'ent': 'BL86'},
  {'start': 93, 'end': 98, 'label': 0, 'type': 'LOC', 'ent': 'Earth'},
  {'start': 99,
   'end': 117,
   'label': 2,
   'type': 'ORDINAL',
   'ent': 'for the first time'}],
 'hallucinations': []},
{'id': 9137,
 'pred': "Labour's Welsh leader has said he still wants Jeremy Corbyn to be prime minister, despite not being a fan of the party's UK leader.",
 'ents': [{'start': 9,
   'end': 14,
   'label': 0,
   'type': 'ORG',
   'ent': 'Welsh'},
  {'start': 46, 'end': 52, 'label': 0, 'type': 'PERSON', 'ent': 'Jeremy'},
  {'start': 53, 'end': 59, 'label': 0, 'type': 'PERSON', 'ent': 'Corbyn'},
  {'start': 121, 'end': 123, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []},
{'id': 3974,
 'pred': 'A man has been shot dead and another wounded by police in the US city of Ferguson, Missouri, a year after the fatal shooting of unarmed black teenager Michael Brown in the town.',
 'ents': [{'start': 62, 'end': 64, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 73, 'end': 81, 'label': 0, 'type': 'GPE', 'ent': 'Ferguson'},
  {'start': 83, 'end': 91, 'label': 1, 'type': 'GPE', 'ent': 'Missouri'},
  {'start': 93, 'end': 99, 'label': 0, 'type': 'DATE', 'ent': 'a year'},
  {'start': 151,
   'end': 158,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Michael'},
  {'start': 159, 'end': 164, 'label': 0, 'type': 'PERSON', 'ent': 'Brown'}],
 'hallucinations': []},
{'id': 6119,
 'pred': '(Close): Stocks on Wall Street closed lower as the dollar rose against the yen and the euro, but shares in Time Warner Cable rose after it agreed to buy Charter Communications.',
 'ents': [{'start': 107,
   'end': 124,
   'label': 3,
   'type': 'ORG',
   'ent': 'Time Warner Cable'},
  {'start': 153,
   'end': 175,
   'label': 3,
   'type': 'ORG',
   'ent': 'Charter Communications'}],
 'hallucinations': []},
{'id': 1759,
 'pred': 'Star Wars and Harry Potter actor Warwick Davis has said his caravan has been stolen, even though it was locked with a hitch and wheel lock.',
 'ents': [{'start': 14,
   'end': 19,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Harry'},
  {'start': 20, 'end': 26, 'label': 0, 'type': 'PERSON', 'ent': 'Potter'},
  {'start': 33, 'end': 40, 'label': 1, 'type': 'PERSON', 'ent': 'Warwick'},
  {'start': 41, 'end': 46, 'label': 0, 'type': 'PERSON', 'ent': 'Davis'}],
 'hallucinations': []},
{'id': 3071,
 'pred': 'Michael Horan has been elected as the new president of the Gaelic Athletic Association (GAA) at the annual congress in Dublin.',
 'ents': [{'start': 0,
   'end': 7,
   'label': 2,
   'type': 'PERSON',
   'ent': 'Michael'},
  {'start': 8, 'end': 13, 'label': 0, 'type': 'PERSON', 'ent': 'Horan'},
  {'start': 55,
   'end': 86,
   'label': 1,
   'type': 'ORG',
   'ent': 'the Gaelic Athletic Association'},
  {'start': 88, 'end': 91, 'label': 0, 'type': 'ORG', 'ent': 'GAA'},
  {'start': 119, 'end': 125, 'label': 0, 'type': 'GPE', 'ent': 'Dublin'}],
 'hallucinations': []},
{'id': 5973,
 'pred': 'The head of HM Revenue and Customs (HMRC), Mark Thompson, has told MPs he would review the tax affairs of footballers and their agents if he was in charge.',
 'ents': [{'start': 12, 'end': 34, 'label': 1, 'type': 'ORG', 'ent': 'HM Revenue and Customs'},
  {'start': 36, 'end': 40, 'label': 0, 'type': 'ORG', 'ent': 'HMRC'},
  {'start': 43, 'end': 47, 'label': 2, 'type': 'PERSON', 'ent': 'Mark'},
  {'start': 48,
   'end': 56,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Thompson'}],
 'hallucinations': []},
{'id': 128,
 'pred': "A new virtual reality tour of Nottingham's caves has been launched to allow visitors to experience the hidden world beneath the city.",
 'ents': [{'start': 30,
   'end': 40,
   'label': 0,
   'type': 'ORG',
   'ent': 'Nottingham'}],
 'hallucinations': []},
{'id': 1505,
 'pred': 'A fishing boat has been towed to safety after getting into difficulty in rough seas off the Anglesey coast.',
 'ents': [{'start': 92,
   'end': 100,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Anglesey'}],
 'hallucinations': []},
{'id': 8960,
 'pred': "Bollywood director Sanjay Bhansali is planning to make a film on the life of Indian boxer Mary Kom, who is from the country's north-eastern state of Manipur.",
 'ents': [{'start': 0,
   'end': 9,
   'label': 0,
   'type': 'ORG',
   'ent': 'Bollywood'},
  {'start': 19, 'end': 25, 'label': 1, 'type': 'PERSON', 'ent': 'Sanjay'},
  {'start': 26, 'end': 34, 'label': 0, 'type': 'PERSON', 'ent': 'Bhansali'},
  {'start': 77, 'end': 83, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
  {'start': 90, 'end': 94, 'label': 0, 'type': 'PERSON', 'ent': 'Mary'},
  {'start': 95, 'end': 98, 'label': 0, 'type': 'PERSON', 'ent': 'Kom'},
  {'start': 149, 'end': 156, 'label': 0, 'type': 'GPE', 'ent': 'Manipur'}],
 'hallucinations': []},
{'id': 4719,
 'pred': 'These are some of the new species of animals and plants discovered in the Mekong delta in South East Asia in the past year.',
 'ents': [{'start': 74,
   'end': 80,
   'label': 0,
   'type': 'GPE',
   'ent': 'Mekong'},
  {'start': 90,
   'end': 105,
   'label': 0,
   'type': 'LOC',
   'ent': 'South East Asia'},
  {'start': 109,
   'end': 122,
   'label': 0,
   'type': 'DATE',
   'ent': 'the past year'}],
 'hallucinations': []},
{'id': 2959,
 'pred': 'A fly-tipper has been fined £1,000 after he dumped more than 1,000 tonnes of rubbish at the side of a road and paid £4,000 to have it cleared up.',
 'ents': [{'start': 29,
   'end': 34,
   'label': 0,
   'type': 'MONEY',
   'ent': '1,000'},
  {'start': 51,
   'end': 73,
   'label': 2,
   'type': 'QUANTITY',
   'ent': 'more than 1,000 tonnes'},
  {'start': 117, 'end': 122, 'label': 0, 'type': 'MONEY', 'ent': '4,000'}],
 'hallucinations': []},
{'id': 5776,
 'pred': 'A woman who admitted killing a poker player in a "pitiless and wicked" attack after being wined and dined by him has been jailed for 20 years.',
 'ents': [{'start': 133,
   'end': 141,
   'label': 2,
   'type': 'DATE',
   'ent': '20 years'}],
 'hallucinations': []},
{'id': 3621,
 'pred': 'Power has been restored to the remaining 10,000 properties left without electricity after a substation went offline in Lancashire, Southern Energy has said.',
 'ents': [{'start': 41,
   'end': 47,
   'label': 0,
   'type': 'CARDINAL',
   'ent': '10,000'},
  {'start': 119,
   'end': 129,
   'label': 0,
   'type': 'GPE',
   'ent': 'Lancashire'},
  {'start': 131,
   'end': 146,
   'label': 2,
   'type': 'ORG',
   'ent': 'Southern Energy'}],
 'hallucinations': []},
{'id': 5522,
 'pred': 'Three teenagers jailed for the murder of a teenager who was stabbed to death in a street attack have had their sentences increased at the Court of Appeal.',
 'ents': [{'start': 0,
   'end': 5,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'Three'},
  {'start': 134,
   'end': 153,
   'label': 0,
   'type': 'ORG',
   'ent': 'the Court of Appeal'}],
 'hallucinations': []},
{'id': 1158,
 'pred': 'Ben Woodburn became the youngest player to score on his debut for Liverpool as the Reds reached the EFL Cup semi-finals for the first time with a hard-fought win over Leeds United at Anfield.',
 'ents': [{'start': 0,
   'end': 3,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Ben'},
  {'start': 4, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': 'Woodburn'},
  {'start': 66, 'end': 75, 'label': 0, 'type': 'ORG', 'ent': 'Liverpool'},
  {'start': 83, 'end': 87, 'label': 0, 'type': 'ORG', 'ent': 'Reds'},
  {'start': 100, 'end': 107, 'label': 0, 'type': 'EVENT', 'ent': 'EFL Cup'},
  {'start': 120, 'end': 138, 'label': 2, 'type': 'ORDINAL', 'ent': 'for the first time'},
  {'start': 167,
   'end': 179,
   'label': 0,
   'type': 'ORG',
   'ent': 'Leeds United'},
  {'start': 183, 'end': 190, 'label': 0, 'type': 'ORG', 'ent': 'Anfield'}],
 'hallucinations': []},
{'id': 7532,
 'pred': 'Former world number one Victoria Azarenka has pulled out of this year\'s US Open because of "ongoing family situation" and will not play at the tournament in New York.',
 'ents': [{'start': 20,
   'end': 23,
   'label': 1,
   'type': 'CARDINAL',
   'ent': 'one'},
  {'start': 24, 'end': 32, 'label': 0, 'type': 'PERSON', 'ent': 'Victoria'},
  {'start': 33, 'end': 41, 'label': 0, 'type': 'PERSON', 'ent': 'Azarenka'},
  {'start': 72, 'end': 79, 'label': 0, 'type': 'EVENT', 'ent': 'US Open'},
  {'start': 157, 'end': 165, 'label': 0, 'type': 'GPE', 'ent': 'New York'}],
 'hallucinations': []},
{'id': 7508,
 'pred': "England women's head coach Mark Sampson has named a 23-strong squad for a two-match friendly tour of Sweden and Norway in January ahead of Euro 2017.",
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'GPE',
   'ent': 'England'},
  {'start': 27, 'end': 31, 'label': 1, 'type': 'PERSON', 'ent': 'Mark'},
  {'start': 32, 'end': 39, 'label': 0, 'type': 'PERSON', 'ent': 'Sampson'},
  {'start': 52, 'end': 54, 'label': 2, 'type': 'CARDINAL', 'ent': '23'},
  {'start': 101, 'end': 107, 'label': 0, 'type': 'GPE', 'ent': 'Sweden'},
  {'start': 112, 'end': 118, 'label': 0, 'type': 'GPE', 'ent': 'Norway'},
  {'start': 122,
   'end': 129,
   'label': 0,
   'type': 'DATE',
   'ent': 'January'},
  {'start': 139,
   'end': 148,
   'label': 1,
   'type': 'LAW',
   'ent': 'Euro 2017'}],
 'hallucinations': []},
{'id': 3385,
 'pred': 'Widnes Vikings can turn their poor start to the Super League season around if they can find a winning streak, says assistant coach Chris Betts.',
 'ents': [{'start': 0,
   'end': 14,
   'label': 0,
   'type': 'ORG',
   'ent': 'Widnes Vikings'},
  {'start': 44,
   'end': 60,
   'label': 1,
   'type': 'ORG',
   'ent': 'the Super League'},
  {'start': 131, 'end': 136, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
  {'start': 137, 'end': 142, 'label': 0, 'type': 'PERSON', 'ent': 'Betts'}],
 'hallucinations': []},
{'id': 8958,
 'pred': 'The US presidential election campaign is in full swing, with Hillary Clinton and Bernie Sanders locked in a tight race for the Democratic nomination.',
 'ents': [{'start': 4, 'end': 6, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 61, 'end': 68, 'label': 0, 'type': 'PERSON', 'ent': 'Hillary'},
  {'start': 69, 'end': 76, 'label': 0, 'type': 'PERSON', 'ent': 'Clinton'},
  {'start': 81, 'end': 87, 'label': 0, 'type': 'PERSON', 'ent': 'Bernie'},
  {'start': 88, 'end': 95, 'label': 0, 'type': 'PERSON', 'ent': 'Sanders'},
  {'start': 127,
   'end': 137,
   'label': 0,
   'type': 'NORP',
   'ent': 'Democratic'}],
 'hallucinations': []},
{'id': 4000,
 'pred': 'Scotland\'s first minister has ruled out an independence referendum in 2017, saying there is "no one who thinks that is the case".',
 'ents': [{'start': 0,
   'end': 8,
   'label': 0,
   'type': 'GPE',
   'ent': 'Scotland'},
  {'start': 11, 'end': 16, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
  {'start': 70, 'end': 74, 'label': 0, 'type': 'DATE', 'ent': '2017'}],
 'hallucinations': []},
{'id': 8354,
 'pred': 'Midfielder Gylfi Sigurdsson believes Swansea City are returning to the "old Swansea City" after a poor start to the season.',
 'ents': [{'start': 0,
   'end': 10,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Midfielder'},
  {'start': 11, 'end': 16, 'label': 1, 'type': 'PERSON', 'ent': 'Gylfi'},
  {'start': 17,
   'end': 27,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Sigurdsson'},
  {'start': 37,
   'end': 49,
   'label': 0,
   'type': 'GPE',
   'ent': 'Swansea City'},
  {'start': 76,
   'end': 88,
   'label': 0,
   'type': 'GPE',
   'ent': 'Swansea City'}],
 'hallucinations': []},
{'id': 8722,
 'pred': 'Bolton Wanderers have agreed a deal to sell the club to former striker Chris Holdsworth for an undisclosed fee, BBC Radio Manchester understands.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Bolton'},
  {'start': 7, 'end': 16, 'label': 0, '0': 'PERSON', 'ent': 'Wanderers'},
  {'start': 71, 'end': 76, 'label': 2, '2': 'PERSON', 'ent': 'Chris'},
  {'start': 77,
   'end': 87,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Holdsworth'},
  {'start': 112,
   'end': 132,
   'label': 0,
   'type': 'ORG',
   'ent': 'BBC Radio Manchester'}],
 'hallucinations': []},
{'id': 7654,
 'pred': "The Republic of Ireland's Department of Agriculture is to sell off about £15m of agricultural loans to a consortium of banks and financial institutions.",
 'ents': [{'start': 0,
   'end': 25,
   'label': 0,
   'type': 'GPE',
   'ent': "The Republic of Ireland's"},
  {'start': 26,
   'end': 51,
   'label': 2,
   'type': 'ORG',
   'ent': 'Department of Agriculture'},
  {'start': 67,
   'end': 77,
   'label': 0,
   'type': 'MONEY',
   'ent': 'about £15m'}],
 'hallucinations': []},
{'id': 613,
 'pred': 'A man has been arrested on suspicion of causing death by dangerous driving after two men were killed in a two-car crash.',
 'ents': [{'start': 81,
   'end': 84,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'two'},
  {'start': 106,
   'end': 113,
   'label': 0,
   'type': 'QUANTITY',
   'ent': 'two-car'}],
 'hallucinations': []},
{'id': 5420,
 'pred': "A majority of members of the Republic of Ireland's Citizens' Assembly have voted in favour of a change in the country's abortion laws.",
 'ents': [{'start': 25,
   'end': 50,
   'label': 0,
   'type': 'GPE',
   'ent': "the Republic of Ireland's"},
  {'start': 51,
   'end': 69,
   'label': 0,
   'type': 'ORG',
   'ent': "Citizens' Assembly"}],
 'hallucinations': []},
{'id': 11150,
 'pred': "BBC Northern Ireland has won three awards at this year's Irish National Film and TV Awards for its television and radio productions.",
 'ents': [{'start': 0,
   'end': 20,
   'label': 0,
   'type': 'ORG',
   'ent': 'BBC Northern Ireland'},
  {'start': 29, 'end': 34, 'label': 0, 'type': 'CARDINAL', 'ent': 'three'},
  {'start': 57,
   'end': 90,
   'label': 2,
   'type': 'ORG',
   'ent': 'Irish National Film and TV Awards'}],
 'hallucinations': []},
{'id': 9338,
 'pred': 'Zambia have been drawn in the same group as hosts South Korea in the group stage of the Fifa Under-20 World Cup for the first time.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'GPE',
   'ent': 'Zambia'},
  {'start': 50, 'end': 61, 'label': 0, 'type': 'GPE', 'ent': 'South Korea'},
  {'start': 88, 'end': 92, 'label': 0, 'type': 'ORG', 'ent': 'Fifa'},
  {'start': 102,
   'end': 111,
   'label': 0,
   'type': 'EVENT',
   'ent': 'World Cup'}],
 'hallucinations': []},
{'id': 8033,
 'pred': 'Mullah Akhtar Mansour, the Afghan Taliban leader who was killed in a US drone strike last week, was a man of few words.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Mullah'},
  {'start': 7, 'end': 13, 'label': 0, 'type': 'PERSON', 'ent': 'Akhtar'},
  {'start': 14, 'end': 21, 'label': 0, 'type': 'PERSON', 'ent': 'Mansour'},
  {'start': 27, 'end': 33, 'label': 0, 'type': 'NORP', 'ent': 'Afghan'},
  {'start': 34, 'end': 41, 'label': 0, 'type': 'ORG', 'ent': 'Taliban'},
  {'start': 69, 'end': 71, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 85, 'end': 94, 'label': 0, 'type': 'DATE', 'ent': 'last week'}],
 'hallucinations': []},
{'id': 2147,
 'pred': 'US economist Gary Hart and Swedish-born economist Carl Holmstrom have won the Nobel prize for economics for their work on public sector contracts.',
 'ents': [{'start': 0, 'end': 2, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 13, 'end': 17, 'label': 2, 'type': 'PERSON', 'ent': 'Gary'},
  {'start': 18, 'end': 22, 'label': 0, 'type': 'PERSON', 'ent': 'Hart'},
  {'start': 27, 'end': 34, 'label': 3, 'type': 'NORP', 'ent': 'Swedish'},
  {'start': 50, 'end': 54, 'label': 2, 'type': 'PERSON', 'ent': 'Carl'},
  {'start': 55,
   'end': 64,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Holmstrom'},
  {'start': 74,
   'end': 89,
   'label': 0,
   'type': 'WORK_OF_ART',
   'ent': 'the Nobel prize'}],
 'hallucinations': []},
{'id': 4799,
 'pred': 'Seven migrants accused of setting fire to a Polish man in a Berlin underground station on Christmas Day are to be charged as adults, German prosecutors say.',
 'ents': [{'start': 0,
   'end': 5,
   'label': 0,
   'type': 'QUANTITY',
   'ent': 'Seven'},
  {'start': 44, 'end': 50, 'label': 0, 'type': 'NORP', 'ent': 'Polish'},
  {'start': 60, 'end': 66, 'label': 0, 'type': 'GPE', 'ent': 'Berlin'},
  {'start': 90,
   'end': 103,
   'label': 0,
   'type': 'DATE',
   'ent': 'Christmas Day'},
  {'start': 133, 'end': 139, 'label': 0, 'type': 'NORP', 'ent': 'German'}],
 'hallucinations': []},
{'id': 403,
 'pred': 'Key quotes from Northern Ireland politicians on the outcome of the assembly election and the future of power sharing.',
 'ents': [{'start': 16,
   'end': 32,
   'label': 0,
   'type': 'GPE',
   'ent': 'Northern Ireland'}],
 'hallucinations': []},
{'id': 1858,
 'pred': 'Chelsea and Manchester City have been charged by the Football Association for failing to control their players following a mass brawl in the Premier League game between the sides on Saturday.',
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'ORG',
   'ent': 'Chelsea'},
  {'start': 12,
   'end': 27,
   'label': 1,
   'type': 'GPE',
   'ent': 'Manchester City'},
  {'start': 49,
   'end': 73,
   'label': 1,
   'type': 'ORG',
   'ent': 'the Football Association'},
  {'start': 137,
   'end': 155,
   'label': 1,
   'type': 'ORG',
   'ent': 'the Premier League'},
  {'start': 182,
   'end': 190,
   'label': 1,
   'type': 'DATE',
   'ent': 'Saturday'}],
 'hallucinations': []},
{'id': 1846,
 'pred': 'A 15-year-old girl who was raped and murdered by a man she met on Facebook was chased and pinned to the ground by another man after she tried to escape from his flat, a court has heard.',
 'ents': [{'start': 66,
   'end': 74,
   'label': 0,
   'type': 'ORG',
   'ent': 'Facebook'}],
 'hallucinations': []},
{'id': 5751,
 'pred': 'The Crown Prosecution Service (CPS) has been criticised by inspectors for not doing enough to help victims of crime in England and Wales.',
 'ents': [{'start': 0,
   'end': 29,
   'label': 0,
   'type': 'ORG',
   'ent': 'The Crown Prosecution Service'},
  {'start': 119, 'end': 126, 'label': 0, 'type': 'GPE', 'ent': 'England'},
  {'start': 131, 'end': 136, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': []},
{'id': 8748,
 'pred': 'Ten men have been arrested in Chad in connection with the rape of an 18-year-old woman in the capital, Chad, in February, police say.',
 'ents': [{'start': 0,
   'end': 3,
   'label': 2,
   'type': 'CARDINAL',
   'ent': 'Ten'},
  {'start': 30, 'end': 34, 'label': 0, 'type': 'GPE', 'ent': 'Chad'},
  {'start': 103, 'end': 107, 'label': 0, 'type': 'GPE', 'ent': 'Chad'},
  {'start': 112,
   'end': 120,
   'label': 0,
   'type': 'DATE',
   'ent': 'February'}],
 'hallucinations': []},
{'id': 5185,
 'pred': "US President Donald Trump's daughter Ivanka Trump has been named a senior White House adviser, the first adult child of a US president to work in the White House.",
 'ents': [{'start': 0, 'end': 2, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 13, 'end': 19, 'label': 1, 'type': 'PERSON', 'ent': 'Donald'},
  {'start': 20, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'},
  {'start': 44, 'end': 49, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'},
  {'start': 74, 'end': 85, 'label': 0, 'type': 'ORG', 'ent': 'White House'},
  {'start': 99, 'end': 104, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
  {'start': 122, 'end': 124, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 146,
   'end': 161,
   'label': 0,
   'type': 'ORG',
   'ent': 'the White House'}],
 'hallucinations': []},
{'id': 5105,
 'pred': 'An Israeli police officer has been placed on forced leave after a video emerged showing him attacking a Palestinian lorry driver in a row over parking.',
 'ents': [{'start': 3,
   'end': 10,
   'label': 0,
   'type': 'NORP',
   'ent': 'Israeli'},
  {'start': 104,
   'end': 115,
   'label': 0,
   'type': 'NORP',
   'ent': 'Palestinian'}],
 'hallucinations': []},
{'id': 3558,
 'pred': 'The man in charge of Titanic Belfast\'s visitors\' centre has said Northern Ireland\'s tourism industry needs to be "mature enough" to make it easy for people to visit the attraction.',
 'ents': [{'start': 21,
   'end': 38,
   'label': 0,
   'type': 'ORG',
   'ent': "Titanic Belfast's"},
  {'start': 65,
   'end': 83,
   'label': 0,
   'type': 'GPE',
   'ent': "Northern Ireland's"}],
 'hallucinations': []},
{'id': 1613,
 'pred': 'A British man accused of murdering a US soldier in Iraq was "directly involved" in making bombs for use in the conflict, a court has heard.',
 'ents': [{'start': 2,
   'end': 9,
   'label': 0,
   'type': 'NORP',
   'ent': 'British'},
  {'start': 37, 'end': 39, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 51, 'end': 55, 'label': 0, 'type': 'GPE', 'ent': 'Iraq'}],
 'hallucinations': []},
{'id': 8158,
 'pred': 'The US Central Intelligence Agency (CIA) has been accused of trying to meddle in the affairs of several African countries over the past 50 years, according to a BBC investigation.',
 'ents': [{'start': 0,
   'end': 34,
   'label': 1,
   'type': 'ORG',
   'ent': 'The US Central Intelligence Agency'},
  {'start': 36, 'end': 39, 'label': 0, 'type': 'ORG', 'ent': 'CIA'},
  {'start': 104, 'end': 111, 'label': 0, 'type': 'NORP', 'ent': 'African'},
  {'start': 127,
   'end': 144,
   'label': 0,
   'type': 'DATE',
   'ent': 'the past 50 years'},
  {'start': 161, 'end': 164, 'label': 0, 'type': 'ORG', 'ent': 'BBC'}],
 'hallucinations': []},
{'id': 3155,
 'pred': "When Leicester City won the Premier League title last season, it was hailed as one of the greatest sporting stories of the 20th Century and helped boost the city's tourism industry.",
 'ents': [{'start': 5,
   'end': 19,
   'label': 0,
   'type': 'GPE',
   'ent': 'Leicester City'},
  {'start': 28,
   'end': 42,
   'label': 0,
   'type': 'ORG',
   'ent': 'Premier League'},
  {'start': 49,
   'end': 60,
   'label': 0,
   'type': 'DATE',
   'ent': 'last season'},
  {'start': 119,
   'end': 135,
   'label': 2,
   'type': 'DATE',
   'ent': 'the 20th Century'}],
 'hallucinations': []},
{'id': 6715,
 'pred': 'Actor Sir Christopher Lee has released a new album of heavy metal songs, which he has described as "the most metal fictional character I know".',
 'ents': [{'start': 10,
   'end': 21,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Christopher'},
  {'start': 22, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Lee'}],
 'hallucinations': []},
{'id': 9410,
 'pred': 'Serial killer Levi Bellfield, who was jailed for life for the rape and murder of 15-year-old Milly Dowler, has denied making a confession to police.',
 'ents': [{'start': 14,
   'end': 18,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Levi'},
  {'start': 19,
   'end': 28,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Bellfield'},
  {'start': 81, 'end': 92, 'label': 2, 'type': 'PERSON', 'ent': '15-year-old'},
  {'start': 93, 'end': 98, 'label': 0, 'type': 'PERSON', 'ent': 'Milly'},
  {'start': 99, 'end': 105, 'label': 0, 'type': 'PERSON', 'ent': 'Dowler'}],
 'hallucinations': []},
{'id': 4560,
 'pred': 'Oscar-winning actress Keira Knightley and British musician James Righton have married in a small ceremony in France.',
 'ents': [{'start': 22,
   'end': 27,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Keira'},
  {'start': 28,
   'end': 37,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Knightley'},
  {'start': 42, 'end': 49, 'label': 1, 'type': 'NORP', 'ent': 'British'},
  {'start': 59, 'end': 64, 'label': 0, 'type': 'PERSON', 'ent': 'James'},
  {'start': 65, 'end': 72, 'label': 0, 'type': 'PERSON', 'ent': 'Righton'},
  {'start': 109, 'end': 115, 'label': 1, 'type': 'GPE', 'ent': 'France'}],
 'hallucinations': []},
{'id': 7199,
 'pred': 'Plans for a £100m shake-up of education in the Jedburgh area of the Scottish Borders have moved a step closer.',
 'ents': [{'start': 12,
   'end': 17,
   'label': 2,
   'type': 'MONEY',
   'ent': '£100m'},
  {'start': 47, 'end': 55, 'label': 0, 'type': 'GPE', 'ent': 'Jedburgh'},
  {'start': 64,
   'end': 84,
   'label': 0,
   'type': 'ORG',
   'ent': 'the Scottish Borders'}],
 'hallucinations': []},
{'id': 8789,
 'pred': 'More than 100 pupils have been sent home from an academy in Manchester after failing to meet uniform standards, the BBC has learned.',
 'ents': [{'start': 0,
   'end': 13,
   'label': 1,
   'type': 'CARDINAL',
   'ent': 'More than 100'},
  {'start': 60, 'end': 70, 'label': 1, 'type': 'GPE', 'ent': 'Manchester'},
  {'start': 116, 'end': 119, 'label': 0, 'type': 'ORG', 'ent': 'BBC'}],
 'hallucinations': []},
{'id': 1263,
 'pred': 'Black voters in the US city of Ferguson have elected their first black council members since a black teenager was shot dead by a white police officer last year.',
 'ents': [{'start': 20, 'end': 22, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 31, 'end': 39, 'label': 0, 'type': 'GPE', 'ent': 'Ferguson'},
  {'start': 150,
   'end': 159,
   'label': 0,
   'type': 'DATE',
   'ent': 'last year'}],
 'hallucinations': []},
{'id': 4408,
 'pred': 'Sweden has become the latest country to experiment with the shortening of the work day, with a series of local and national trials targeting employees with high levels of stress and burnout.',
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'GPE',
   'ent': 'Sweden'}],
 'hallucinations': []},
{'id': 8196,
 'pred': 'Tributes have been paid to Joel Richards, one of the British victims of the Tunisia beach attack, who was killed in the attack.',
 'ents': [{'start': 27,
   'end': 31,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Joel'},
  {'start': 32, 'end': 40, 'label': 0, 'type': 'PERSON', 'ent': 'Richards'},
  {'start': 53, 'end': 60, 'label': 1, 'type': 'NORP', 'ent': 'British'},
  {'start': 76, 'end': 83, 'label': 0, 'type': 'GPE', 'ent': 'Tunisia'}],
 'hallucinations': []},
{'id': 3237,
 'pred': 'Manchester City manager Pep Guardiola says Barcelona are the best team he has played against in his managerial career, and says they are "the best team in the Premier League".',
 'ents': [{'start': 0,
   'end': 15,
   'label': 1,
   'type': 'GPE',
   'ent': 'Manchester City'},
  {'start': 24, 'end': 27, 'label': 1, 'type': 'PERSON', 'ent': 'Pep'},
  {'start': 28,
   'end': 37,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Guardiola'},
  {'start': 43, 'end': 52, 'label': 0, 'type': 'GPE', 'ent': 'Barcelona'},
  {'start': 155,
   'end': 173,
   'label': 0,
   'type': 'ORG',
   'ent': 'the Premier League'}],
 'hallucinations': []},
{'id': 5089,
 'pred': 'A former NHS nurse who downloaded hundreds of thousands of indecent images of children has been jailed for six years.',
 'ents': [{'start': 9, 'end': 12, 'label': 0, 'type': 'ORG', 'ent': 'NHS'},
  {'start': 34,
   'end': 55,
   'label': 0,
   'type': 'CARDINAL',
   'ent': 'hundreds of thousands'},
  {'start': 107,
   'end': 116,
   'label': 2,
   'type': 'DATE',
   'ent': 'six years'}],
 'hallucinations': []},
{'id': 9283,
 'pred': 'Aberdeen children from the 1950s have been invited to take part in a major study of family health in Scotland as part of a new project.',
 'ents': [{'start': 0,
   'end': 8,
   'label': 0,
   'type': 'ORG',
   'ent': 'Aberdeen'},
  {'start': 23, 'end': 32, 'label': 0, 'type': 'DATE', 'ent': 'the 1950s'},
  {'start': 101, 'end': 109, 'label': 0, 'type': 'GPE', 'ent': 'Scotland'}],
 'hallucinations': []},
{'id': 5466,
 'pred': 'Wrexham are interested in signing Plymouth Argyle striker Jordan Harvey after he turned down a new deal with the League Two club.',
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Wrexham'},
  {'start': 34,
   'end': 49,
   'label': 1,
   'type': 'ORG',
   'ent': 'Plymouth Argyle'},
  {'start': 58, 'end': 64, 'label': 2, 'type': 'PERSON', 'ent': 'Jordan'},
  {'start': 65, 'end': 71, 'label': 0, 'type': 'PERSON', 'ent': 'Harvey'},
  {'start': 113,
   'end': 123,
   'label': 1,
   'type': 'ORG',
   'ent': 'League Two'}],
 'hallucinations': []},
{'id': 655,
 'pred': 'Aston Villa have signed winger Scott Sinclair from Manchester City for an undisclosed fee on a three-and-a-half-year deal, subject to international clearance.',
 'ents': [{'start': 0,
   'end': 5,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Aston'},
  {'start': 6, 'end': 11, 'label': 0, 'type': 'PERSON', 'ent': 'Villa'},
  {'start': 31, 'end': 36, 'label': 1, 'type': 'PERSON', 'ent': 'Scott'},
  {'start': 37, 'end': 45, 'label': 0, 'type': 'PERSON', 'ent': 'Sinclair'},
  {'start': 51,
   'end': 66,
   'label': 0,
   'type': 'GPE',
   'ent': 'Manchester City'},
  {'start': 95,
   'end': 116,
   'label': 2,
   'type': 'CARDINAL',
   'ent': 'three-and-a-half-year'}],
 'hallucinations': []},
{'id': 260,
 'pred': 'Ethnic minority children in Hong Kong are still being denied equal access to education, according to a new report by the Equal Opportunities Commission.',
 'ents': [{'start': 28,
   'end': 37,
   'label': 0,
   'type': 'GPE',
   'ent': 'Hong Kong'},
  {'start': 117,
   'end': 151,
   'label': 0,
   'type': 'ORG',
   'ent': 'the Equal Opportunities Commission'}],
 'hallucinations': []},
{'id': 6356,
 'pred': 'European Commission President Jean-Claude Juncker has set out his proposals for dealing with the migrant crisis in a speech to MEPs, saying Europe cannot house all the misery in the world.',
 'ents': [{'start': 0,
   'end': 29,
   'label': 1,
   'type': 'ORG',
   'ent': 'European Commission President'},
  {'start': 30,
   'end': 41,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Jean-Claude'},
  {'start': 42, 'end': 49, 'label': 0, 'type': 'PERSON', 'ent': 'Juncker'},
  {'start': 140, 'end': 146, 'label': 0, 'type': 'LOC', 'ent': 'Europe'}],
 'hallucinations': []},
{'id': 2348,
 'pred': 'Police in the US state of Missouri have fired tear gas and used water cannon to break up a protest against a curfew imposed after the killing of a black teenager by a police officer.',
 'ents': [{'start': 14, 'end': 16, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 26, 'end': 34, 'label': 1, 'type': 'GPE', 'ent': 'Missouri'}],
 'hallucinations': []},
{'id': 8363,
 'pred': 'A man who died after being hit by a car in Caterham, south-west London, may have been involved in an earlier stabbing, an inquest has heard.',
 'ents': [{'start': 43,
   'end': 51,
   'label': 0,
   'type': 'GPE',
   'ent': 'Caterham'},
  {'start': 64, 'end': 70, 'label': 1, 'type': 'GPE', 'ent': 'London'}],
 'hallucinations': []},
{'id': 4616,
 'pred': 'A motorcyclist has been killed in a collision with a car in South Yorkshire.',
 'ents': [{'start': 60,
   'end': 75,
   'label': 0,
   'type': 'GPE',
   'ent': 'South Yorkshire'}],
 'hallucinations': []},
{'id': 5873,
 'pred': "Donald Trump's campaign manager, Corey Lewandowski, has been charged over an alleged assault on a reporter at a rally, his campaign has said.",
 'ents': [{'start': 0,
   'end': 6,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Donald'},
  {'start': 7, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': "Trump"},
  {'start': 33, 'end': 38, 'label': 1, 'type': 'PERSON', 'ent': 'Corey'},
  {'start': 39,
   'end': 50,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Lewandowski'}],
 'hallucinations': []},
{'id': 185,
 'pred': 'Firefighters are continuing to tackle a blaze at a wood chip recycling plant in Bridgend county which has destroyed up to 2,000 tonnes of wood chip.',
 'ents': [{'start': 80,
   'end': 95,
   'label': 0,
   'type': 'GPE',
   'ent': 'Bridgend county'},
  {'start': 116,
   'end': 134,
   'label': 0,
   'type': 'QUANTITY',
   'ent': 'up to 2,000 tonnes'}],
 'hallucinations': []},
{'id': 2436,
 'pred': 'Former Arkansas governor and Republican presidential candidate Mike Huckabee has announced he will run for the White House in 2016, eight years after he failed to win the Republican nomination.',
 'ents': [{'start': 7,
   'end': 15,
   'label': 0,
   'type': 'GPE',
   'ent': 'Arkansas'},
  {'start': 29, 'end': 39, 'label': 0, 'type': 'NORP', 'ent': 'Republican'},
  {'start': 63, 'end': 67, 'label': 0, 'type': 'PERSON', 'ent': 'Mike'},
  {'start': 68, 'end': 76, 'label': 0, 'type': 'PERSON', 'ent': 'Huckabee'},
  {'start': 107,
   'end': 122,
   'label': 0,
   'type': 'ORG',
   'ent': 'the White House'},
  {'start': 126, 'end': 130, 'label': 0, 'type': 'DATE', 'ent': '2016'},
  {'start': 132,
   'end': 143,
   'label': 0,
   'type': 'DATE',
   'ent': 'eight years'},
  {'start': 171,
   'end': 181,
   'label': 0,
   'type': 'NORP',
   'ent': 'Republican'}],
 'hallucinations': []},
{'id': 3385,
 'pred': 'Widnes Vikings can turn their poor start to the Super League season around if they can find a winning streak, says assistant coach Chris Betts.',
 'ents': [{'start': 0,
   'end': 14,
   'label': 0,
   'type': 'ORG',
   'ent': 'Widnes Vikings'},
  {'start': 44,
   'end': 60,
   'label': 1,
   'type': 'ORG',
   'ent': 'the Super League'},
  {'start': 131, 'end': 136, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
  {'start': 137, 'end': 142, 'label': 0, 'type': 'PERSON', 'ent': 'Betts'}],
 'hallucinations': []},
{'id': 2070,
 'pred': 'Kilmarnock have re-signed former Hearts midfielder Jamie Hamill, who will make his return to the club for the first time since his departure in January.',
 'ents': [{'start': 0,
   'end': 10,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Kilmarnock'},
  {'start': 51, 'end': 56, 'label': 0, 'type': 'PERSON', 'ent': 'Jamie'},
  {'start': 57, 'end': 63, 'label': 0, 'type': 'PERSON', 'ent': 'Hamill'},
  {'start': 144, 'end': 151, 'label': 2, 'type': 'DATE', 'ent': 'January'}],
 'hallucinations': []}
]

In [54]:
labels = []
total_length = 0
for a in annotated:
    for ent in a['ents']:
        assert a['pred'][ent['start']: ent['end']] == ent['ent'], '{} - {}'.format(ent['ent'], a['pred'][ent['start']: ent['start']])

In [55]:
print(len(annotated))

200


In [74]:
ID = randint(0, len(xsum_preds))
ID = 6126
print(ID)

6126


In [75]:
new_dict = {}
new_dict['id'] = ID
new_dict['pred'] = xsum_preds[ID]
new_dict['ents'] = []

for ent in nlp(new_dict['pred']).to_json()['ents']:
    if ent['label'] != 'PERSON':
        new_dict['ents'].append(ent)
    else:
        name_parts = new_dict['pred'][ent['start']: ent['end']].split()
        init_start = ent['start']
        for p in name_parts:
            new_dict['ents'].append({'start': init_start, 'end': init_start + len(p), 'label': 'PERSON'})
            init_start = init_start + len(p) + 1
            
for e in new_dict['ents']:
    e['type'] = e['label']
    e['ent'] = new_dict['pred'][e['start']: e['end']]
    e['label'] = None

new_dict['hallucinations'] = []

In [76]:
new_dict

{'id': 6126,
 'pred': 'The Supreme Court has ruled that the assembly can set minimum wages for agricultural workers in Wales, the Welsh government has said.',
 'ents': [{'start': 0,
   'end': 17,
   'label': None,
   'type': 'ORG',
   'ent': 'The Supreme Court'},
  {'start': 96, 'end': 101, 'label': None, 'type': 'GPE', 'ent': 'Wales'},
  {'start': 107, 'end': 112, 'label': None, 'type': 'ORG', 'ent': 'Welsh'}],
 'hallucinations': []}

In [77]:
for i, e in enumerate(new_dict['ents']):
    print('{} - {}'.format(i, new_dict['pred'][e['start']: e['end']]))

print('- target:')
print(xsum_target[ID])
print('\n- prediction:')
print(xsum_preds[ID])
print('\n- source:')
print(xsum_source[ID])

0 - The Supreme Court
1 - Wales
2 - Welsh
- target:
Welsh ministers can protect the wages of 13,000 farm workers, in light of a ruling by the Supreme Court.

- prediction:
The Supreme Court has ruled that the assembly can set minimum wages for agricultural workers in Wales, the Welsh government has said.

- source:
It comes after a dispute over whether Wales had the power to set minimum wages, or if that right laid with the UK government. UK Attorney General Dominic Grieve originally blocked a law passed by AMs last year, saying it was not within the assembly's powers. UK ministers said they were grateful for clarity on the issue. The dispute arose after the assembly passed a law last year to set minimum wages, and the terms and conditions, of agricultural workers in Wales after the UK government abolished the Agricultural Wages Board, which had regulated the sector. On Wednesday, First Minister Carwyn Jones said the "significant" Supreme Court judgment "goes some way to clarifying the

In [78]:
# with open('annotated.json', 'w') as fout:
#     json.dump(annotated , fout)

#### Statistics 

In [61]:
labels = []
for a in annotated:
    for e in a['ents']:
        labels.append(e['label'])

In [62]:
len(labels)

723

In [63]:
for l in set(labels):
    print(labels.count(l))
    print(labels.count(l) / len(labels))

498
0.6887966804979253
120
0.16597510373443983
89
0.12309820193637622
16
0.022130013831258646


In [64]:
len('European Commission President')

29